<br><hr><br>

# Eunicea spp. Microbiome Analysis

This is a synopsis of the bioinformatic processing of microbiome data collected in the doctoral work of Adriana Sarmiento. Microbiome samples of the coral genus *Eunicea* spp. were collected 2019 in the Caribbean Sea and sequenced at Iowa State University.
<br>This Jupyter-Notebook is available at
https://github.com/julianselke/Eunicea_spp_Microbiome_Analysis .<br><br>

Maintainer: Julian Selke<br>
Contact: julianselke@web.de<br>
Last updated: 28.05.2022<br><br>

<hr>

# Setup

This notebook comprises the entire analysis of the *Eunicea* spp. microbiome dataset. It somewhat still presents a medley of various scripts produced as analysis progressed. One inconsistency remains: Some functions in ```R``` require a transposed _feature table_. If any error occurs, transposing the respective _feature table_ might present a quick fix. 

In order to run ```R``` commands in this notebook, either the ```R```-kernel has to be running or ```rpy2``` has to be loaded.

In [ ]:
%load_ext rpy2.ipython

Now, ```%%R``` in a cell enables *R* code to be translated to ```python``` and to be run, and ```%%bash``` enables *bash* code to be run. <br>
However, some functions cannot be translated and therefore require a running ```R```-kernel. The kernel has to be changed manually and you will be advised when to do so.

Check current workind directory. Please note that ```!``` at the beginning of a line initiates the ```sh``` shell, not ```bash```.

In [ ]:
! realpath .

Check if conda environment is working correctly.

In [ ]:
%%bash

source activate qiime2-2022.2
qiime --help

Line numbers can be displayed by pressing ```SHIFT + L``` when no cell is activated.

To adjust plotting options of ```R``` cells add and run the follwing cell: 

```options(repr.plot.width = X, repr.plot.height = Y, repr.plot.res = Z)```

Replace ```X```, ```Y```, and ```Z``` with desired values.

Press ```Shift```+```O``` to see all output of the selected cell. Useful, e.g., when tables are cut off. 

<hr>

# Content

[fastp](#fastp) <br>
[Qiime2](#Qiime2) <br>
[Preparation](#Preparation) <br>
[Exploration](#Exploration) <br>
> [Euclidian Distance](#EuclidianDistance) <br>
[Bray-Curtis Dissimilarity](#Bray-CurtisDissimilarity) <br>
[Jaccard Distance](#JaccardDistance) <br>
[Unweighted UniFrac Distance](#UnweightedUniFracDistance) <br>
[Weighted UniFrac Distance](#WeightedUniFracDistance) <br>
[Aitchison Distance](#AitchisonDistance) <br>
[PhILR Distance](#PhILRDistance) <br>

[Statistics](#Statistics) <br>
[t-SNE](#t-SNE) <br>
[Composition](#Composition) <br>
[iNEXT](#iNEXT) <br>
[Core Microbiome](#CoreMicrobiome) <br>
[Host Phylogeny](#HostPhylogeny) <br>
[ALDEx2](#ALDEx2) <br>

<br><hr><br>


# fastp

Preprocessing with fastp enables base correction in overlapping regions based on PHRED scores. 

In [ ]:
%%bash

echo '

#!/usr/bin/env bash

# Default values of arguments
OUTPUT_DIRECTORY=./fastp_output

# Loop through arguments and process them
for arg in "$@"
do
    case $arg in
        -i|--input_directory)
	# get list of all files in directory
        INPUT_DIRECTORY=$2
        shift # Remove --input_directory from processing
        ;;
        -o|--output_directory)
	OUTPUT_DIRECTORY=$3
        shift # Remove --output_directory from processing
	;;
        -h|--help)
	echo
	printf "\033[1;32m  Process files of a directory with fastp. \033[0m\n"
        echo
        echo "  Syntax: bash $0 [-i -o|-h]"
        echo
        echo "  options:"
        echo
        echo "   \033[1;30m-i     The input directory containing fastq files."
        echo "          \033[0mdefault: \e[47m.\e[49m"
        echo
        echo "   \033[1;30m-o     The output directory to save processed files to."
        echo "          \033[0mdefault: \e[47m./fastp_output\e[49m" 
        echo
        echo "   \033[1;30m-h     Print this Help."
        echo
	exit 0
    esac
done

INPUT_FILES_PATH=$(find $INPUT_DIRECTORY -type f | sed "s/R._001\.fastq\.gz//g" | uniq)

mkdir $OUTPUT_DIRECTORY

for file in $INPUT_FILES_PATH
do
  echo "Processing $file"
  fastp -i "$file"R1_001.fastq.gz \
        -I "$file"R2_001.fastq.gz \
        -o "$OUTPUT_DIRECTORY"/$(basename $file)R1_001.fastq.gz \
        -O "$OUTPUT_DIRECTORY"/$(basename $file)R2_001.fastq.gz -c  
done

' > preprocess_fastp.sh 

Give permission to execute script.

In [ ]:
! chmod +x preprocess_fastp.sh 

See help options of script. See also: https://github.com/OpenGene/fastp

In [ ]:
! ./preprocess_fastp.sh -h

JSON and HTML reports are overwritten in each run. 

In [ ]:
! ./preprocess_fastp.sh -i ./raw_data

<br><hr><br>

# Qiime2

[&#8593; back to content &#8593;](#Content)

### manifest

Write the manifest file to import forward and reverse sequences into Qiime2. 

In [ ]:
%%bash

ls -p /root/biommar/fastp_output/ | grep -v / | \
awk -F'_' 'BEGIN {OFS=""; print "sample-id,absolute-filepath,direction"} \
{ if ($0 ~ /.+_R1_.+/) {print $1,",","/root/biommar/fastp_output/",$0,",forward" \
} else {print $1,",","/root/biommar/fastp_output/",$0,",reverse"}}' \
> /root/biommar/data/eunicea_PE_manifest

### Import raw data

In [ ]:
%%bash

source activate qiime2-2022.2

qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /root/biommar/data/eunicea_PE_manifest \
  --output-path /root/biommar/data/eunicea_paired_end_demux.qza \
  --input-format PairedEndFastqManifestPhred33

### Trimming

Doc.: https://docs.qiime2.org/2022.2/plugins/available/dada2/denoise-paired/

Denoise and dereplicate the sequences, filters chimeras and merge the paired-end sequences.

In [ ]:
%%bash

source activate qiime2-2022.2

qiime dada2 denoise-paired \
  --i-demultiplexed-seqs /root/biommar/data/eunicea_paired_end_demux.qza \
  --p-trunc-len-f 230 \
  --p-trunc-len-r 165 \
  --p-n-threads 4 \
  --o-table /root/biommar/data/dada_table.qza \
  --o-representative-sequences /root/biommar/data/dada_rep_seqs.qza \
  --o-denoising-stats /root/biommar/data/dada_stats.qza

### Filtering

Filter features to at leat 10 occurrences across all samples and sequnces based on featurs subsequently.

In [ ]:
%%bash

source activate qiime2-2022.2

qiime feature-table filter-features \
  --i-table /root/biommar/data/dada_table.qza \
  --p-min-frequency 10 \
  --o-filtered-table /root/biommar/data/dada_filtered_table.qza

qiime feature-table filter-seqs \
  --i-data /root/biommar/data/dada_rep_seqs.qza\
  --i-table /root/biommar/data/dada_filtered_table.qza \
  --o-filtered-data /root/biommar/data/dada_filtered_rep_seqs.qza

### Phylogeny

Fragment Insertion

Doc.: https://library.qiime2.org/plugins/q2-fragment-insertion/16/

Paper: SEPP: SATé-Enabled Phylogenetic Placement

Phylogenetic placement of retained sequences.
In order to explore the phylogenetic composition of our samples, we place the probes into a prebuild phylogenetic tree. The utilized tool, SEPP, has proven to be advantagious in working with sOTUs, since it is using a controlled reference tree, but also allows non-perfect hits to its tips.
The phylogeny of the sample may offer biologically relevant information and allows are more sophisticated comparision of microbial communities.


<hr>
Download the reference database.

In [ ]:
%cd /root/biommar/data/
! wget https://data.qiime2.org/2022.2/common/sepp-refs-silva-128.qza

<hr> 

Check if any errors occurred while downloading. The warning ``` no properly formatted MD5 checksum lines found ``` can be ignored. You should see an **OK** in the second line.

In [ ]:
%%bash 

md5sum -c <<<"7879792a6f42c5325531de9866f5c4de  /root/biommar/data/sepp-refs-silva-128.qza" \
- /root/biommar/data/sepp-refs-silva-128.qza | tail -n -1

<hr>

Run the fragment insertion.

In [ ]:
%%bash

source activate qiime2-2022.2

qiime fragment-insertion sepp \
  --i-representative-sequences /root/biommar/data/dada_filtered_rep_seqs.qza  \
  --i-reference-database /root/biommar/data/sepp-refs-silva-128.qza \
  --p-threads 4 \
  --o-tree /root/biommar/data/sepp_phylogeny.qza \
  --o-placements /root/biommar/data/sepp_placements.qza

### --> not enough RAM (16GB insufficient)
--> run on cluster instead

### Read Classification

Doc.: https://docs.qiime2.org/2022.2/plugins/available/feature-classifier/classify-sklearn/

Classification of reads by taxon using a pre-fitted classifier.
Without the knowledge of the species, genera, etc. present in the samples, the biological interpretation of the communities is very limited. As an example, no mutualist/pathogen relations can be estimated, probale metabolic processes and therefore biological function of the species remain veiled. Furthermore, this can inform us about contamination still present in the samples, e.g. species present in the human microbiome but not in the soil may indicate contamination of respective samples while collecting or processing.

<hr>
Download the prefitted classifier.

In [ ]:
%cd /root/biommar/data/
! wget https://data.qiime2.org/2022.2/common/silva-138-99-515-806-nb-classifier.qza

<hr> 

Check if any errors occurred while downloading. The warning ``` no properly formatted MD5 checksum lines found ``` can be ignored. You should see an **OK** in the second line.

In [ ]:
%%bash 

md5sum -c <<<"e05afad0fe87542704be96ff483824d4  /root/biommar/data/silva-138-99-515-806-nb-classifier.qza" \
- /root/biommar/data/silva-138-99-515-806-nb-classifier.qza | tail -n -1

In [ ]:
%%bash

source activate qiime2-2022.2

qiime feature-classifier classify-sklearn \
  --i-classifier /root/biommar/data/silva-138-99-515-806-nb-classifier.qza \
  --i-reads /root/biommar/data/dada_filtered_rep_seqs.qza \
  --o-classification /root/biommar/data/filtered_taxonomy_silva.qza

### --> not enough RAM (16GB insufficient)
--> run on cluster instead

<hr>

Filter features to match metadata.

Doc.: https://docs.qiime2.org/2021.11/plugins/available/feature-table/filter-samples/

In [ ]:
%%bash

source activate qiime2-2022.2

qiime feature-table filter-samples \
  --i-table /root/biommar/data/dada_filtered_table.qza \
  --m-metadata-file /root/biommar/data/Metadata_Eunicea_Bacteria.tsv \
  --o-filtered-table /root/biommar/data/dada_filtered_table_metadata_filtered.qza

### Filter out Chloroplast and Mitochindria

Doc.: https://docs.qiime2.org/2022.2/plugins/available/feature-table/filter-features/

Contamination belonging to these two taxa/organelles is present in our data and blurring the distance metrics. A SQLite command is used to specify the filter applied on taxa.


In [ ]:
%%bash

source activate qiime2-2022.2

qiime feature-table filter-features \
  --i-table /root/biommar/data/dada_filtered_table_metadata_filtered.qza  \
  --m-metadata-file /root/biommar/data/filtered_taxonomy_silva.qza \
  --p-where  "(Taxon NOT LIKE '%c__Chloroplast%') AND (Taxon NOT LIKE '%f__mitochondria%')" \
  --o-filtered-table /root/biommar/data/dada_table_no_mitochloro_silva.qza

### Extract Data

In [ ]:
%%bash

source activate qiime2-2022.2

cd  /root/biommar/data
unzip -qq dada_table_no_mitochloro_silva.qza
cp $(echo $(ls -td -- */ | head -n 1) $(echo "data/feature-table.biom") | sed 's/\ //g') .
biom convert -i feature-table.biom -o feature-table.tsv --to-tsv
rm -r $(ls -td -- */ | head -n 1)
unzip -qq filtered_taxonomy_silva.qza
cp $(echo $(ls -td -- */ | head -n 1) $(echo "data/taxonomy.tsv") | sed 's/\ //g') . 
rm -r $(ls -td -- */ | head -n 1)
unzip -qq sepp_phylogeny.qza 
cp $(echo $(ls -td -- */ | head -n 1) $(echo "data/tree.nwk") | sed 's/\ //g') .
rm -r $(ls -td -- */ | head -n 1)


<br><hr><br><br><hr><br>

# Analysis in R

<span style="color:red">**PLEASE CHANGE KERNEL ACCORDINGLY!**</span>

Select ```Kernel``` in the menu bar at the top, click on ```Change kernel```, and select ```R```.

<br><hr><br><br><hr><br>

In [ ]:
library( ALDEx2,        quietly = TRUE)
library( ape,           quietly = TRUE)
library( aplot,         quietly = TRUE)
library( cluster,       quietly = TRUE)
library( ggdendro,      quietly = TRUE)
library( grid,          quietly = TRUE)
library( glmnet,        quietly = TRUE)
library( ggplot2,       quietly = TRUE)
library( ggfortify,     quietly = TRUE)
library( ggthemes,      quietly = TRUE)
library( gridExtra,     quietly = TRUE)
library( ggpmisc,       quietly = TRUE)
library( ggsci,         quietly = TRUE)
library( ggtree,        quietly = TRUE)
library( tidyr,         quietly = TRUE)
library( metagenomeSeq, quietly = TRUE)
library( microbiome,    quietly = TRUE)
library( phyloseq,      quietly = TRUE)
library( philr,         quietly = TRUE)
library( mia,           quietly = TRUE)
library( vegan,         quietly = TRUE)
library( multcompView,  quietly = TRUE)
library( plyr,          quietly = TRUE)
library( dplyr,         quietly = TRUE)
library( Rtsne,         quietly = TRUE)
library( plotly,        quietly = TRUE)
library( zgtools,       quietly = TRUE)
library( reshape,       quietly = TRUE)
library( onewaytests,   quietly = TRUE)
library( iNEXT,         quietly = TRUE)
library( phytools,      quietly = TRUE)
library( reshape2,      quietly = TRUE)
library( repr,          quietly = TRUE)
library( kableExtra,    quietly = TRUE)
library( IRdisplay,     quietly = TRUE)
library( jsonlite,      quietly = TRUE)

<br><hr><br>

# Preparation

[&#8593; back to content &#8593;](#Content)

### Import Data

In [ ]:
bac_feature_table <- data.frame(t(read.delim("/root/biommar/data/feature-table.tsv",
                                             skip        = 1, 
                                             row.names   = 1, 
                                             check.names = FALSE)),
                                check.names = FALSE)

bac_metadata <- read.table("/root/biommar/data/Metadata_Eunicea_Bacteria.tsv",
                           header      = T, 
                           sep         = "\t", 
                           check.names = FALSE)[, 1:12]

bac_taxonomy <- read.table("/root/biommar/data/taxonomy.tsv",
                           header    = T,
                           row.names = 1,
                           sep       = "\t")

# filter out outgroups
bac_metadata <- bac_metadata[!bac_metadata$genus %in% c("Water sample", "Muricea"),]

# assign row names as sample ID to facilitate merging on row names
rownames(bac_metadata) <- bac_metadata$sampleid

# make sure to exclude features from other objects as well
bac_feature_table <- bac_feature_table[rownames(bac_feature_table) %in% rownames(bac_metadata),]

bac_taxonomy <- bac_taxonomy[rownames(bac_taxonomy) %in% colnames(bac_feature_table),]

<hr>

### Prepare Data

In [ ]:
# filter taxonomy to remove discarded ASVs (Chloroplasts and Mitochondria)
# !!! asymmetric operation !!! setdiff(b, a) != setdiff(a, b)
out <- setdiff(rownames(bac_taxonomy), colnames(bac_feature_table))
# remove missing samples
# bac_taxonomy <- bac_taxonomy[-c(which(rownames(bac_taxonomy) %in% out)), ]

# same for metadata
out <- setdiff(rownames(bac_metadata), rownames(bac_feature_table)) 
# remove missing samples
bac_metadata <- bac_metadata[-c(which(rownames(bac_metadata) %in% out)), ]

# make all entries numeric
bac_feature_table <- mutate_all(bac_feature_table, function(x) as.numeric(as.character(x)))

# split taxonomy
taxa <- separate(
  data = bac_taxonomy,
  col  = Taxon,
  into = c(
    "kingdom",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species"
  ),
  sep = ";"
)

# store row names
taxa_names <- rownames(taxa)
# denote all cells without information as "unidentified" 
taxa[is.na(taxa)] <- "unidentified"
taxa <- as.data.frame(lapply(taxa, function(x) gsub("[a-z]__$", "unidentified", x)))
# crop leading pattern
taxa <- as.data.frame(lapply(taxa, function(x) gsub("[a-z]__", "", x)))
# remove leading whitespaces
taxa <- as.data.frame(sapply(taxa, function(x) {gsub("^ ", "", x)}))
# re-assign row names
rownames(taxa) <- taxa_names
# cleanup
rm(taxa_names)

write.table(taxa[,1:7],
            file  = "/root/biommar/data/taxa.txt",
            sep   = "\t",
            quote = F)

<hr>

### Filtering

In [ ]:
# redundant, since this filtering has been done in Qiime2 before
# subset the feature table to contain only features that have at least ten reads
bac_feature_table <- bac_feature_table[, (colSums(bac_feature_table) > 9)]

bac_taxonomy <- bac_taxonomy[rownames(bac_taxonomy) %in% colnames(bac_feature_table), ]

# subset taxa as well
taxa <- taxa[rownames(taxa) %in% colnames(bac_feature_table), ]

# filter data: remove samples with <1000 reads
NumReads <- rowSums(bac_feature_table)
# remove respective samples
bac_feature_table <- bac_feature_table[NumReads > 1000, ]
# cleanup
rm(NumReads)

# make sure filtered features are excluded
bac_metadata <- bac_metadata[rownames(bac_metadata) %in% rownames(bac_feature_table),]

bac_taxonomy <- bac_taxonomy[rownames(bac_taxonomy) %in% colnames(bac_feature_table),]

taxa <- 
  taxa[rownames(taxa) %in% colnames(bac_feature_table),]

# calculate sparcity, i.e. percentage of cells with zero count
sparcity <- length(bac_feature_table[bac_feature_table == 0]) / 
  (nrow(bac_feature_table) * ncol(bac_feature_table)) * 100

print("Sparcity, i.e. percentage of cells in feature table with zero-count is:")
print(sparcity)

# filter for taxa without occurrences in remaining samples 
taxa_bool <- ifelse(bac_feature_table > 0, 1, 0)
# aggregate by feature
taxa_occ <- as.vector(colSums(taxa_bool))
taxa_occ_abs <- as.vector(colSums(bac_feature_table))

# remove respective taxa
bac_feature_table <- bac_feature_table[, taxa_occ > 0]

# aggregate by sample
sample_occ <- as.vector(rowSums(taxa_bool))
sample_occ_abs <- as.vector(rowSums(bac_feature_table))

# keep only samples with more than one feature
bac_feature_table <- bac_feature_table[sample_occ > 1, ]

<hr>

### Plotting

In [ ]:
# set option number display
options(scipen=999)
options(repr.plot.width = 4, repr.plot.height = 4, repr.plot.res = 300)

In [ ]:
plot_taxa_occ <- ggplot(data.frame(taxa_occ), aes(x = taxa_occ)) + 
  geom_histogram(color = "gray10", fill = "#00468BFF", bins = 30) + 
  scale_y_log10() +
  theme_bw() +
  ylab("Number of Features") +
  xlab("Occurrences [Number of Samples]") +
  ggtitle("Feature Occurrences") +
  theme(plot.title = element_text(hjust=0.5))

plot_taxa_occ

In [ ]:
plot_taxa_occ_abs <- ggplot(data.frame(taxa_occ_abs), aes(x = taxa_occ_abs)) + 
  geom_histogram(color = "gray10", fill = "#00468BFF", bins = 30) + 
  scale_x_log10(breaks = c(1,10,100,1000,10000,100000,1000000)) +
  theme_bw() +
  ylab("Number of Features") +
  xlab("Count across all Samples") +
  ggtitle("Feature Counts") +
  theme(plot.title = element_text(hjust=0.5))

plot_taxa_occ_abs

In [ ]:
plot_taxa_occ_abs <- ggplot(data.frame(taxa_occ_abs), aes(x = taxa_occ_abs)) + 
  geom_histogram(color = "gray10", fill = "#00468BFF", bins = 30) + 
  scale_x_log10(breaks = c(1,10,100,1000,10000,100000,1000000)) +
  theme_bw() +
  ylab("Number of Features") +
  xlab("Count across all Samples") +
  ggtitle("Feature Counts") +
  theme(plot.title = element_text(hjust=0.5))

plot_taxa_occ_abs 

In [ ]:
plot_sample_occ_abs <- ggplot(data.frame(sample_occ_abs), aes(x = sample_occ_abs)) + 
  geom_histogram(color = "gray10", fill = "#00468BFF", bins = 30) + 
  scale_x_log10() +
  theme_bw() +
  xlab("Count of Features") +
  ylab("Number of Samples") +
  ggtitle("Feature Counts per Sample") +
  theme(plot.title = element_text(hjust=0.5))


plot_sample_occ_abs

<hr>

### Write Files

In [ ]:
write.table(bac_feature_table,
            file      = "/root/biommar/data/B_asv.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

write.table(bac_metadata,
            file      = "/root/biommar/data/B_asv_metadata.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

write.table(bac_taxonomy,
            file      = "/root/biommar/data/B_asv_taxonomy.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

<hr>

### Cumulative Sum Scaling (CSS) normalization to remove biases in count data

In [ ]:
# create a MRexperiment object from count data.
table_metagenomeseq = newMRexperiment(t(bac_feature_table))
# calculate the percentile for which to sum counts up to and scale by
pthq = cumNormStatFast(table_metagenomeseq)
# calculate each column's quantile and calculate the sum up to and 
# including that quantile
table_metagenomeseq_norm = cumNorm(table_metagenomeseq, p = pthq)

bac_feature_table_css <- data.frame(MRcounts(table_metagenomeseq_norm, norm=TRUE))
bac_feature_table_css <- as.data.frame(t(bac_feature_table_css))

# cleanup
rm(pthq, table_metagenomeseq_norm, table_metagenomeseq)

# make sure to exclude features from other objects as well
bac_taxonomy_css <- bac_taxonomy[rownames(bac_taxonomy) %in% rownames(bac_feature_table_css),]

bac_metadata_css <- bac_metadata[rownames(bac_metadata) %in% colnames(bac_feature_table_css),]

bac_taxa_css <- taxa[rownames(taxa) %in% rownames(bac_feature_table_css),]

<hr>

### Write CSS Files

In [ ]:
write.table(bac_feature_table_css,
            file      = "/root/biommar/data/CSS_B_asv.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

write.table(bac_metadata_css,
            file      = "/root/biommar/data/CSS_B_asv_metadata.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

write.table(bac_taxonomy_css,
            file      = "/root/biommar/data/CSS_B_asv_taxonomy.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

<hr>

### PhILR 

In [ ]:
otu.file  <- file.path("/root/biommar/data/B_asv.txt")
tax.file  <- file.path("/root/biommar/data/taxa.txt")
meta.file <- file.path("/root/biommar/data/B_asv_metadata.txt")

bac_ps <- read_phyloseq(
  otu.file      = otu.file,
  taxonomy.file = tax.file,
  metadata.file = meta.file,
  type          = "simple",
  sep           = "\t"
)

meta <- read.table(meta.file, row.names = 1, header = T, sep="\t")

# import tree
tree <- read_tree("/root/biommar/data/tree.nwk")

bac_ps <- merge_phyloseq(bac_ps, tree)

tse <- mia::makeTreeSummarizedExperimentFromPhyloseq(bac_ps)

In [ ]:
tse

<hr>

### Filter Extremely Low-Abundance OTUs

Prevalence is a measurement, which describes in how many samples certain
microbes were detected.

A 0.01% relative abundance threshold is set. 

In [ ]:
tse <-  tse %>% subsetByPrevalentTaxa(
  detection   = 0.01/100,
  prevalence  = 0,
  as_relative = TRUE)

# Collapse the tree
# Otherwise the original tree with all nodes is kept
# (including those that were filtered out from rowData)
tree <- ape::keep.tip(phy = rowTree(tse), 
                      tip = rowLinks(tse)$nodeNum)
rowTree(tse) <- tree

## Add a new assay with a pseudocount 
assays(tse)$counts.shifted <- assays(tse)$counts + 1e-12

# enumerate nodes
#tree <- makeNodeLabel(tree, method="number", prefix='n')

# check if tree is rooted and if tree is binary
if(!is.rooted(tree)){ 
  system("echo '\033[1;31mError: Phylogenetic tree is not rooted!\033[0m'")}
if(!is.binary(tree)){ 
  system("echo '\033[1;31mError: Phylogenetic tree is not binary!\033[0m'")}
# Add the modified tree back to the (`TreeSE`) data object 
rowTree(tse) <- tree

bac_feature_table_philr <- as.data.frame(philr(tse, 
                                               part.weights = 'uniform', 
                                               ilr.weights  = 'uniform', 
                                               return.all   = FALSE, 
                                               abund_values = "counts.shifted"))

<hr>

### Write PhILR Files

In [ ]:
write.table(bac_feature_table_philr,
            file      = "/root/biommar/data/PhILR_B_asv.txt",
            sep       = "\t",
            row.names = T,
            quote     = F)

<br><hr><br>

# Exploration

[&#8593; back to content &#8593;](#Content)


## Remark:
<br>
The following distance and dissimilarity metrics are applied to count data of samples.

Two filters were applied a priori: ASVs with less than 10 counts across all samples and samples with less than 1000 reads where excluded.

Where indicated, counts where transformed by **C**umulative **S**um **S**caling.<br>

https://doi.org/10.1038/nmeth.2658

<hr>

<a id='EuclidianDistance'></a>

<br><hr><br>
# Euclidian Distance

[&#8593; back to content &#8593;](#Content)


The euclidian distance between two points is defined as
<br><br>
$$d_e(p,q) = \sqrt{\sum_{i=1}^{n}(p_i-q_i)^2}$$
<br>
where *n* is the number of dimensions.



In [ ]:
bac_feature_table <- read.delim("/root/biommar/data/B_asv.txt", check.names = F)
bac_metadata <- read.delim("/root/biommar/data/B_asv_metadata.txt")

# transpose and set class of data for computation
bac_feature_table <- as.data.frame(bac_feature_table, check.names = F)
bac_feature_table <- mutate_all(bac_feature_table, 
                                    function(x) as.numeric(as.character(x)))

bac_feature_table_css <- read.delim("/root/biommar/data/CSS_B_asv.txt", check.names = F)
bac_metadata_css <- read.delim("/root/biommar/data/CSS_B_asv_metadata.txt")

# transpose and set class of data for computation
bac_feature_table_css <- as.data.frame(t(bac_feature_table_css), check.names = F)
bac_feature_table_css <- mutate_all(bac_feature_table_css, 
                                    function(x) as.numeric(as.character(x)))

# calculate distance matrix
bac_euclidian_dist <- vegan::vegdist(bac_feature_table, method = "euclidian")

# compute the PCoA
bac_euclidian_pcoa <- ape::pcoa(bac_euclidian_dist)

# save PCo vector data for plot
bac_euclidian_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_euclidian_pcoa$vectors)),
    X = bac_euclidian_pcoa$vectors[, 1],
    Y = bac_euclidian_pcoa$vectors[, 2],
    Y = bac_euclidian_pcoa$vectors[, 3]
  )

# add metadata
bac_euclidian_pcoa_data$Sample <- rownames(bac_euclidian_pcoa_data)

colnames(bac_euclidian_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_euclidian_pcoa_data <-
  left_join(bac_euclidian_pcoa_data, bac_metadata, by = "sampleid")
                                    
axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[1]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[2]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[3]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

##### Remark: 
If necessary, please adjust ```width``` and ```height``` based on your screens dimensions.<br>
Uncomment ```%>% zgtools::plotly_elegant()``` to view in dark theme.

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_euclidian_pcoa_data,
    x      = ~X,
    y      = ~Y,
    z      = ~Z,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Euclidian Distances") #%>% zgtools::plotly_elegant()
                                    
embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_euclidian_nmds_2D <- metaMDS(bac_feature_table,k=2,trymax=20,distance="euclidian")
bac_euclidian_nmds_3D <- metaMDS(bac_feature_table,k=3,trymax=20,distance="euclidian")

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7, repr.plot.res = 200)

cat("\n", "Stress 3D: ", bac_euclidian_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_euclidian_nmds_2D[["stress"]])

stressplot(bac_euclidian_nmds_3D)

In [ ]:
bac_euclidian_nmds_3D_plot_data <- as.data.frame(bac_euclidian_nmds_3D[["points"]])

bac_euclidian_nmds_3D_plot_data$sampleid <- rownames(bac_euclidian_nmds_3D_plot_data)

bac_euclidian_nmds_3D_plot_data <-
  left_join(bac_euclidian_nmds_3D_plot_data, bac_metadata, by = "sampleid")

axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_euclidian_nmds_3D_plot_data$values$Relative_eig[[1]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_euclidian_nmds_3D_plot_data$values$Relative_eig[[2]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_euclidian_nmds_3D_plot_data$values$Relative_eig[[3]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_euclidian_nmds_3D_plot_data,
    x      = ~MDS1,
    y      = ~MDS2,
    z      = ~MDS3,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nNMDS on Euclidian Distances") #%>% zgtools::plotly_elegant()
                                    
embed_notebook(p)

<hr>

### Cumulative Sum Scaling

In [ ]:
# calculate distance matrix
bac_euclidian_dist <- vegan::vegdist(bac_feature_table_css, method = "euclidian")

# compute the PCoA
bac_euclidian_pcoa <- ape::pcoa(bac_euclidian_dist)

# save PCo vector data for plot
bac_euclidian_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_euclidian_pcoa$vectors)),
    X = bac_euclidian_pcoa$vectors[, 1],
    Y = bac_euclidian_pcoa$vectors[, 2],
    Z = bac_euclidian_pcoa$vectors[, 3]
  )

# add metadata
bac_euclidian_pcoa_data$Sample <- rownames(bac_euclidian_pcoa_data)

colnames(bac_euclidian_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_euclidian_pcoa_data <-
  left_join(bac_euclidian_pcoa_data, bac_metadata_css, by = "sampleid")
  
axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[1]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[2]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_euclidian_pcoa$values$Relative_eig[[3]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_euclidian_pcoa_data,
    x      = ~X,
    y      = ~Y,
    z      = ~Z,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    text   = bac_euclidian_pcoa_data$sampleid,
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Euclidian Distances [CSS]") #%>% zgtools::plotly_elegant()
                                    
embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_euclidian_nmds_2D <- metaMDS(bac_feature_table_css,k=2,trymax=20,distance="euclidian")
bac_euclidian_nmds_3D <- metaMDS(bac_feature_table_css,k=3,trymax=20,distance="euclidian")

In [ ]:
cat("\n", "Stress 3D: ", bac_euclidian_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_euclidian_nmds_2D[["stress"]])

stressplot(bac_euclidian_nmds_3D)

In [ ]:
bac_euclidian_nmds_3D_plot_data <- as.data.frame(bac_euclidian_nmds_3D[["points"]])

bac_euclidian_nmds_3D_plot_data$sampleid <- rownames(bac_euclidian_nmds_3D_plot_data)

bac_euclidian_nmds_3D_plot_data <-
  left_join(bac_euclidian_nmds_3D_plot_data, bac_metadata, by = "sampleid")

theme = list(tickfont=list(color=c('#323232')), color = "#aaaaaa")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_euclidian_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Euclidian Distances [CSS]")

embed_notebook(p)


<br><hr><br>

<a id='Bray-CurtisDissimilarity'></a>

# Bray-Curtis Dissimilarity

[&#8593; back to content &#8593;](#Content)

The Bray-Curtis dissimilarity between two sites is defined as
<br><br>
$$BC(p,q) = 1-\frac{2 C_{pq}}{S_p+S_q}$$
<br>
where $C_{pq}$ is the sum of the lesser values for only those species in common between both sites. ${S_p}$ and ${S_q}$ are the total number of specimens counted at both sites. 
<br><br>
https://doi.org/10.2307/1942268


In [ ]:
# calculate distance matrix
bac_braycurtis_dist <- vegan::vegdist(bac_feature_table, method = "bray")

# compute the PCoA
bac_braycurtis_pcoa <- ape::pcoa(bac_braycurtis_dist)

# save PCo vector data for plot
bac_braycurtis_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_braycurtis_pcoa$vectors)),
    X = bac_braycurtis_pcoa$vectors[, 1],
    Y = bac_braycurtis_pcoa$vectors[, 2],
    Z = bac_braycurtis_pcoa$vectors[, 3]
  )

# add metadata
bac_braycurtis_pcoa_data$Sample <- rownames(bac_braycurtis_pcoa_data)

colnames(bac_braycurtis_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_braycurtis_pcoa_data <-
  left_join(bac_braycurtis_pcoa_data, bac_metadata, by = "sampleid")

axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_braycurtis_pcoa$values$Relative_eig[[1]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_braycurtis_pcoa$values$Relative_eig[[2]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
                                    
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_braycurtis_pcoa$values$Relative_eig[[3]] * 100, 2), 
                             "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_braycurtis_pcoa_data,
    x      = ~X,
    y      = ~Y,
    z      = ~Z,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    text   = bac_braycurtis_pcoa_data$sampleid,
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Bray-Curtis Dissimilarities") #%>% zgtools::plotly_elegant()
                                    
embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_braycurtis_nmds_2D <- metaMDS(bac_feature_table,k=2,trymax=20,distance="bray")
bac_braycurtis_nmds_3D <- metaMDS(bac_feature_table,k=3,trymax=20,distance="bray")

In [ ]:
cat("\n", "Stress 3D: ", bac_braycurtis_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_braycurtis_nmds_2D[["stress"]])

stressplot(bac_braycurtis_nmds_3D)

In [ ]:
bac_braycurtis_nmds_3D_plot_data <- as.data.frame(bac_braycurtis_nmds_3D[["points"]])

bac_braycurtis_nmds_3D_plot_data$sampleid <- rownames(bac_braycurtis_nmds_3D_plot_data)

bac_braycurtis_nmds_3D_plot_data <-
  left_join(bac_braycurtis_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_braycurtis_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Bray-Curtis Dissimilarities")

embed_notebook(p)

<hr>

### Cumulative Sum Scaling

In [ ]:
# calculate distance matrix
bac_braycurtis_dist <- vegan::vegdist(bac_feature_table_css, method = "bray")

# compute the PCoA
bac_braycurtis_pcoa <- ape::pcoa(bac_braycurtis_dist)

# save PCo vector data for plot
bac_braycurtis_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_braycurtis_pcoa$vectors)),
    X = bac_braycurtis_pcoa$vectors[, 1],
    Y = bac_braycurtis_pcoa$vectors[, 2],
    Z = bac_braycurtis_pcoa$vectors[, 3]
  )

# add metadata
bac_braycurtis_pcoa_data$Sample <- rownames(bac_braycurtis_pcoa_data)

colnames(bac_braycurtis_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_braycurtis_pcoa_data <-
  left_join(bac_braycurtis_pcoa_data, bac_metadata, by = "sampleid")

# plotting
axx <- list(title = paste("PCo 1 - ", round(
  bac_braycurtis_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axy <- list(title = paste("PCo 2 - ", round(
  bac_braycurtis_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axz <- list(title = paste("PCo 3 - ", round(
  bac_braycurtis_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_braycurtis_pcoa_data,
    x = ~X,
    y = ~Y,
    z = ~Z,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=axx,yaxis=axy,zaxis=axz),
         title = "\nPCoA on Bray-Curtis Dissimilarities [CSS]")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_braycurtis_nmds_2D <- metaMDS(bac_feature_table_css,k=2,trymax=20,distance="bray")
bac_braycurtis_nmds_3D <- metaMDS(bac_feature_table_css,k=3,trymax=20,distance="bray")

In [ ]:
cat("\n", "Stress 3D: ", bac_braycurtis_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_braycurtis_nmds_2D[["stress"]])

stressplot(bac_braycurtis_nmds_3D)

In [ ]:
bac_braycurtis_nmds_3D_plot_data <- as.data.frame(bac_braycurtis_nmds_3D[["points"]])

bac_braycurtis_nmds_3D_plot_data$sampleid <- rownames(bac_braycurtis_nmds_3D_plot_data)

bac_braycurtis_nmds_3D_plot_data <-
  left_join(bac_braycurtis_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_braycurtis_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Bray-Curtis Dissimilarities [CSS]")

embed_notebook(p)

<br><hr><br>
<a id='JaccardDistance'></a>

# Jaccard Distance

[&#8593; back to content &#8593;](#Content)

The Jaccard distance between two sites is defined as
<br><br>
$$d_j(P,Q) = \frac{|P \cup Q| - |P \cap Q|}{P \cap Q}$$
<br>
i.e. by dividing the difference of the sizes of the union and the intersection of two sets by the size of the union.
<br><br>
https://doi.org/10.1111/j.1469-8137.1912.tb05611.x

In [ ]:
# calculate distance matrix
bac_jaccard_dist <- vegan::vegdist(bac_feature_table, method = "jaccard")

# compute the PCoA
bac_jaccard_pcoa <- ape::pcoa(bac_jaccard_dist)

# save PCo vector data for plot
bac_jaccard_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_jaccard_pcoa$vectors)),
    X = bac_jaccard_pcoa$vectors[, 1],
    Y = bac_jaccard_pcoa$vectors[, 2],
    Z = bac_jaccard_pcoa$vectors[, 3]
  )

# add metadata
bac_jaccard_pcoa_data$Sample <- rownames(bac_jaccard_pcoa_data)

colnames(bac_jaccard_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_jaccard_pcoa_data <-
  left_join(bac_jaccard_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title = paste("PCo 1 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axy <- list(title = paste("PCo 2 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axz <- list(title = paste("PCo 3 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_jaccard_pcoa_data,
    x = ~X,
    y = ~Y,
    z = ~Z,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=axx,yaxis=axy,zaxis=axz),
         title = "\nPCoA on Jaccard Distances")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_jaccard_nmds_2D <- metaMDS(bac_feature_table,k=2,trymax=20,distance="jaccard")
bac_jaccard_nmds_3D <- metaMDS(bac_feature_table,k=3,trymax=20,distance="jaccard")

In [ ]:
cat("\n", "Stress 3D: ", bac_jaccard_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_jaccard_nmds_2D[["stress"]])

stressplot(bac_jaccard_nmds_3D)

In [ ]:
bac_jaccard_nmds_3D_plot_data <- as.data.frame(bac_jaccard_nmds_3D[["points"]])

bac_jaccard_nmds_3D_plot_data$sampleid <- rownames(bac_jaccard_nmds_3D_plot_data)

bac_jaccard_nmds_3D_plot_data <-
  left_join(bac_jaccard_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_jaccard_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Jaccard Distances")

embed_notebook(p)

<hr>

### Cumulative Sum Scaling

In [ ]:
# calculate distance matrix
bac_jaccard_dist <- vegan::vegdist(bac_feature_table_css, method = "jaccard")

# compute the PCoA
bac_jaccard_pcoa <- ape::pcoa(bac_jaccard_dist)

# save PCo vector data for plot
bac_jaccard_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_jaccard_pcoa$vectors)),
    X = bac_jaccard_pcoa$vectors[, 1],
    Y = bac_jaccard_pcoa$vectors[, 2],
    Z = bac_jaccard_pcoa$vectors[, 3]
  )

# add metadata
bac_jaccard_pcoa_data$Sample <- rownames(bac_jaccard_pcoa_data)

colnames(bac_jaccard_pcoa_data) <- c("sampleid", "X", "Y","Z")

bac_jaccard_pcoa_data <-
  left_join(bac_jaccard_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title = paste("PCo 1 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')),color = "#aaaaaa")
axy <- list(title = paste("PCo 2 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axz <- list(title = paste("PCo 3 - ", round(
  bac_jaccard_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_jaccard_pcoa_data,
    x = ~X,
    y = ~Y,
    z = ~Z,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=axx,yaxis=axy,zaxis=axz),
         title = "\nPCoA on Jaccard Distances [CSS]")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_jaccard_nmds_2D <- metaMDS(bac_feature_table_css,k=2,trymax=20,distance="jaccard")
bac_jaccard_nmds_3D <- metaMDS(bac_feature_table_css,k=3,trymax=20,distance="jaccard")

In [ ]:
cat("\n", "Stress 3D: ", bac_jaccard_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_jaccard_nmds_2D[["stress"]])

stressplot(bac_jaccard_nmds_3D)

In [ ]:
bac_jaccard_nmds_3D_plot_data <- as.data.frame(bac_jaccard_nmds_3D[["points"]])

bac_jaccard_nmds_3D_plot_data$sampleid <- rownames(bac_jaccard_nmds_3D_plot_data)

bac_jaccard_nmds_3D_plot_data <-
  left_join(bac_jaccard_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_jaccard_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Jaccard Distances [CSS]") 

embed_notebook(p)

<br><hr><br>
<a id='UnweightedUniFracDistance'></a>

# Unweighted UniFrac Distance

[&#8593; back to content &#8593;](#Content)

The unweighted UniFrac distance between two samples is defined as
<br><br>
$$ d^U=\sum_{i=1}^{n} {\frac{b_i|I(p_i^A>0)-I(p_i^B>0)|}{\sum_{i=1}^{n}b_i}} $$
<br>
where $n$ is the number of branches in a rooted phylogenetic tree, $b_i$ is the length of branch $i$, $p_i^A$ and $p_i^B$ are the taxa proportions descending from the branch $i$ for community $A$ and $B$, respectively.  $I(.)$ is the indicator function and only presence/absence of species of branch $i$, $I(p_i^A > 0)$ and $I(p_i^B > 0)$.

More generalized it can be described as
<br><br>
$$ {\displaystyle \left({\frac {sum~of~unshared~branch~lengths}{sum~of~all~tree~branch~lengths}}\right){=}fraction~of~total~unshared~branch~lengths} $$
<br><br>
https://doi.org/10.1093/bioinformatics/bts342

In [ ]:
otu.file <- file.path("/root/biommar/data/B_asv.txt")
tax.file <- file.path("/root/biommar/data/B_asv_taxonomy.txt")
meta.file <- file.path("/root/biommar/data/B_asv_metadata.txt")

bac_ps <- read_phyloseq(
  otu.file      = otu.file,
  taxonomy.file = tax.file,
  metadata.file = meta.file,
  type          = "simple",
  sep           = "\t"
)

# import tree
tree <- read_tree("/root/biommar/data/tree.nwk")

bac_ps <- merge_phyloseq(bac_ps, tree)

bac_u_unifrac_dist <- UniFrac(bac_ps, weighted = FALSE, normalized = TRUE,
  parallel = FALSE, fast = TRUE)
  
# compute the PCoA
bac_u_unifrac_pcoa <- ape::pcoa(bac_u_unifrac_dist)

# save PCo vector data for plot
bac_u_unifrac_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_u_unifrac_pcoa$vectors)),
    X = bac_u_unifrac_pcoa$vectors[, 1],
    Y = bac_u_unifrac_pcoa$vectors[, 2],
    Z = bac_u_unifrac_pcoa$vectors[, 3]
  )

# add metadata
bac_u_unifrac_pcoa_data$Sample <- rownames(bac_u_unifrac_pcoa_data)

colnames(bac_u_unifrac_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_u_unifrac_pcoa_data <-
  left_join(bac_u_unifrac_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title = paste("PCo 1 - ", round(
  bac_u_unifrac_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')),color = "#aaaaaa")
axy <- list(title = paste("PCo 2 - ", round(
  bac_u_unifrac_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axz <- list(title = paste("PCo 3 - ", round(
  bac_u_unifrac_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_u_unifrac_pcoa_data,
    x = ~X,
    y = ~Y,
    z = ~Z,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=axx,yaxis=axy,zaxis=axz),
         title = "\nPCoA on Unweighted UniFrac Distances")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_u_unifrac_nmds_2D <- metaMDS(bac_u_unifrac_dist,k=2,trymax=20)
bac_u_unifrac_nmds_3D <- metaMDS(bac_u_unifrac_dist,k=3,trymax=20)

In [ ]:
cat("\n", "Stress 3D: ", bac_u_unifrac_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_u_unifrac_nmds_2D[["stress"]])

stressplot(bac_u_unifrac_nmds_3D)

In [ ]:
bac_u_unifrac_nmds_3D_plot_data <- as.data.frame(bac_u_unifrac_nmds_3D[["points"]])

bac_u_unifrac_nmds_3D_plot_data$sampleid <- rownames(bac_u_unifrac_nmds_3D_plot_data)

bac_u_unifrac_nmds_3D_plot_data <-
  left_join(bac_u_unifrac_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_nmds_3D_plot_data,
    x          = ~MDS1,
    y          = ~MDS2,
    z          = ~MDS3,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = theme,
                      yaxis = theme,
                      zaxis = theme),
         title = "\nNMDS on Unweighted UniFrac Distances") 

embed_notebook(p)

<hr>

### Cumulative Sum Scaling

In [ ]:
otu.file <- file.path("/root/biommar/data/CSS_B_asv.txt")
tax.file <- file.path("/root/biommar/data/CSS_B_asv_taxonomy.txt")
meta.file <- file.path("/root/biommar/data/CSS_B_asv_metadata.txt")

bac_ps_css <- read_phyloseq(
  otu.file      = otu.file,
  taxonomy.file = tax.file,
  metadata.file = meta.file,
  type          = "simple",
  sep           = "\t"
)

# import tree
tree <- read_tree("/root/biommar/data/tree.nwk")

bac_ps_css <- merge_phyloseq(bac_ps_css, tree)

bac_u_unifrac_dist <- UniFrac(bac_ps_css, weighted = FALSE, normalized = TRUE,
  parallel = FALSE, fast = TRUE)

# compute the PCoA
bac_u_unifrac_pcoa <- ape::pcoa(bac_u_unifrac_dist)

# save PCo vector data for plot
bac_u_unifrac_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_u_unifrac_pcoa$vectors)),
    X = bac_u_unifrac_pcoa$vectors[, 1],
    Y = bac_u_unifrac_pcoa$vectors[, 2],
    Z = bac_u_unifrac_pcoa$vectors[, 3]
  )

# add metadata
bac_u_unifrac_pcoa_data$Sample <- rownames(bac_u_unifrac_pcoa_data)

colnames(bac_u_unifrac_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_u_unifrac_pcoa_data <- left_join(bac_u_unifrac_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_pcoa_data,
    x          = ~X,
    y          = ~Y,
    z          = ~Z,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Unweighted UniFrac Distances [CSS]") 

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_u_unifrac_nmds_2D <- metaMDS(bac_u_unifrac_dist, k = 2, trymax = 20)
bac_u_unifrac_nmds_3D <- metaMDS(bac_u_unifrac_dist, k = 3, trymax = 20)

In [ ]:
cat("\n", "Stress 3D: ", bac_u_unifrac_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_u_unifrac_nmds_2D[["stress"]])

stressplot(bac_u_unifrac_nmds_3D)

In [ ]:
bac_u_unifrac_nmds_3D_plot_data <- as.data.frame(bac_u_unifrac_nmds_3D[["points"]])

bac_u_unifrac_nmds_3D_plot_data$sampleid <- rownames(bac_u_unifrac_nmds_3D_plot_data)

bac_u_unifrac_nmds_3D_plot_data <-left_join(bac_u_unifrac_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_nmds_3D_plot_data,
    x          = ~MDS1,
    y          = ~MDS2,
    z          = ~MDS3,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = theme,
                      yaxis = theme,
                      zaxis = theme),
         title = "\nNMDS on Unweighted UniFrac Distances [CSS]")

embed_notebook(p)

<br><hr><br>
<a id='WeightedUniFracDistance'></a>

# Weighted UniFrac Distance

[&#8593; back to content &#8593;](#Content)

The weighted UniFrac distance between two samples $(A,B)$ is defined as
<br><br>
$$ d^W={\frac{\sum_{i=1}^{n}b_i|p_i^A-p_i^B|}{\sum_{i=1}^{n}b_i(p_i^A+p_i^B)}} $$
<br><br>
https://doi.org/10.1093/bioinformatics/bts342

In [ ]:
otu.file  <- file.path("/root/biommar/data/B_asv.txt")
tax.file  <- file.path("/root/biommar/data/B_asv_taxonomy.txt")
meta.file <- file.path("/root/biommar/data/B_asv_metadata.txt")

bac_ps <- read_phyloseq(
  otu.file      = otu.file,
  taxonomy.file = tax.file,
  metadata.file = meta.file,
  type          = "simple",
  sep           = "\t"
)

# import tree
tree <- read_tree("/root/biommar/data/tree.nwk")

bac_ps <- merge_phyloseq(bac_ps, tree)

bac_u_unifrac_dist <- UniFrac(bac_ps, weighted = TRUE, normalized = TRUE, parallel = FALSE, fast = TRUE)
  
# compute the PCoA
bac_u_unifrac_pcoa <- ape::pcoa(bac_u_unifrac_dist)

# save PCo vector data for plot
bac_u_unifrac_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_u_unifrac_pcoa$vectors)),
    X      = bac_u_unifrac_pcoa$vectors[, 1],
    Y      = bac_u_unifrac_pcoa$vectors[, 2],
    Z      = bac_u_unifrac_pcoa$vectors[, 3]
  )

# add metadata
bac_u_unifrac_pcoa_data$Sample <- rownames(bac_u_unifrac_pcoa_data)

colnames(bac_u_unifrac_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_u_unifrac_pcoa_data <- left_join(bac_u_unifrac_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_u_unifrac_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_pcoa_data,
    x          = ~X,
    y          = ~Y,
    z          = ~Z,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Weighted UniFrac Distances")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_u_unifrac_nmds_2D <- metaMDS(bac_u_unifrac_dist, k = 2, trymax = 20)
bac_u_unifrac_nmds_3D <- metaMDS(bac_u_unifrac_dist, k = 3, trymax = 20)

In [ ]:
cat("\n", "Stress 3D: ", bac_u_unifrac_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_u_unifrac_nmds_2D[["stress"]])

stressplot(bac_u_unifrac_nmds_3D)

In [ ]:
bac_u_unifrac_nmds_3D_plot_data <- as.data.frame(bac_u_unifrac_nmds_3D[["points"]])

bac_u_unifrac_nmds_3D_plot_data$sampleid <- rownames(bac_u_unifrac_nmds_3D_plot_data)

bac_u_unifrac_nmds_3D_plot_data <- left_join(bac_u_unifrac_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_nmds_3D_plot_data,
    x          = ~MDS1,
    y          = ~MDS2,
    z          = ~MDS3,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = theme,
                      yaxis = theme,
                      zaxis = theme),
         title = "NMDS on Weighted UniFrac Distances") %>% 
  zgtools::plotly_elegant()

embed_notebook(p)

<hr>

### Cumulative Sum Scaling

In [ ]:
bac_w_unifrac_dist <- UniFrac(bac_ps_css, weighted = TRUE, normalized = TRUE, parallel = FALSE, fast = TRUE)

# compute the PCoA
bac_w_unifrac_pcoa <- ape::pcoa(bac_w_unifrac_dist)

# save PCo vector data for plot
bac_w_unifrac_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_w_unifrac_pcoa$vectors)),
    X      = bac_w_unifrac_pcoa$vectors[, 1],
    Y      = bac_w_unifrac_pcoa$vectors[, 2],
    Z      = bac_w_unifrac_pcoa$vectors[, 3]
  )

# add metadata
bac_w_unifrac_pcoa_data$Sample <- rownames(bac_w_unifrac_pcoa_data)

colnames(bac_w_unifrac_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_w_unifrac_pcoa_data <- left_join(bac_w_unifrac_pcoa_data, bac_metadata, by = "sampleid")

# plotting
axx <- list(title    = paste("PCo 1 - ", round( bac_w_unifrac_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axy <- list(title    = paste("PCo 2 - ", round(bac_w_unifrac_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axz <- list(title    = paste("PCo 3 - ", round(bac_w_unifrac_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_w_unifrac_pcoa_data,
    x          = ~X,
    y          = ~Y,
    z          = ~Z,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "PCoA on Weighted UniFrac Distances [CSS]")

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_u_unifrac_nmds_2D <- metaMDS(bac_w_unifrac_dist,k=2,trymax=20)
bac_u_unifrac_nmds_3D <- metaMDS(bac_w_unifrac_dist,k=3,trymax=20)

In [ ]:
cat("\n", "Stress 3D: ", bac_u_unifrac_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_u_unifrac_nmds_2D[["stress"]])

stressplot(bac_u_unifrac_nmds_3D)

In [ ]:
bac_u_unifrac_nmds_3D_plot_data <- as.data.frame(bac_u_unifrac_nmds_3D[["points"]])

bac_u_unifrac_nmds_3D_plot_data$sampleid <- rownames(bac_u_unifrac_nmds_3D_plot_data)

bac_u_unifrac_nmds_3D_plot_data <-
  left_join(bac_u_unifrac_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_u_unifrac_nmds_3D_plot_data,
    x          = ~MDS1,
    y          = ~MDS2,
    z          = ~MDS3,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = theme,
                      yaxis = theme,
                      zaxis = theme),
         title = "NMDS on Weighted UniFrac Distances [CSS]") 

embed_notebook(p)

### Remark:<br>

Weighted UniFrac distance incorporates the branch lengths as absolute positive values. The phylogenetic tree on which it operates is produced in *Qiime2* with the </span><span class="code">fragment-insertion</span><span class="x"> plugin. Internally, SEPP places the reads on a phylogenetic reference tree (</span><span class="code">silva128</span><span class="x">). Unknown clades may be placed with considerable branch lengths and therefore dominate feature segregation in weighted UniFrac plots.

https://doi.org/10.1128/mSystems.00021-18

<br><hr><br>
<a id='AitchisonDistance'></a>

# Aitchison Distance

[&#8593; back to content &#8593;](#Content)

The Aitchison distance between two $n$-dimensional compositions, $x_i$ and $x_j$ is defined as
<br><br>
$$ d^A(x_i,x_j)=\sqrt{\sum_{g=1}^n\displaystyle \left[ln\frac{x_{gi}}{g(x_i)}-ln\frac{x_{gj}}{g(x_j)}\right]^2} $$
<br>
where $g(.)$ is the geometric mean. The Aitchison distance is equal to the Euclidean distance between clr-transformed compositions. It has scale invariance, perturbation invariance, permutation invariance and sub-compositional dominance.

https://doi.org/10.1093/bioinformatics/bty175

In [ ]:
aitchison_data <- bac_feature_table

# add pseudo count 
aitchison_data[aitchison_data == 0] <- 1e-12

aitchison_data <- t(apply(aitchison_data, 1, 
                          function(x)log(x/exp(mean(log(x))))))
bac_aitchison_dist <- as.matrix(dist(aitchison_data))

# compute the PCoA
bac_aitchison_pcoa <- ape::pcoa(bac_aitchison_dist)

# save PCo vector data for plot
bac_aitchison_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_aitchison_pcoa$vectors)),
    X = bac_aitchison_pcoa$vectors[, 1],
    Y = bac_aitchison_pcoa$vectors[, 2],
    Z = bac_aitchison_pcoa$vectors[, 3]
  )

# add metadata
bac_aitchison_pcoa_data$Sample <- rownames(bac_aitchison_pcoa_data)

colnames(bac_aitchison_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_aitchison_pcoa_data <-
  left_join(bac_aitchison_pcoa_data, bac_metadata, by = "sampleid")

# plotting
axx <- list(title = paste("PCo 1 - ", round(
  bac_aitchison_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axy <- list(title = paste("PCo 2 - ", round(
  bac_aitchison_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")
axz <- list(title = paste("PCo 3 - ", round(
  bac_aitchison_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
  tickfont=list(color=c('#323232')), color = "#aaaaaa")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data       = bac_aitchison_pcoa_data,
    x          = ~X,
    y          = ~Y,
    z          = ~Z,
    color      = ~department,
    colors     = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker     = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "PCoA on Aitchison Distances")
                          
embed_notebook(p)

In [ ]:
p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_aitchison_pcoa_data,
    x      = ~X,
    y      = ~Y,
    z      = ~Z,
    color  = ~Clade,
    colors = c("dodgerblue2", "#E31A1C", "green4", "#6A3D9A", 
               "#FF7F00", "gold1", "maroon", "orchid1", "deeppink1", 
               "blue1", "steelblue4", "darkturquoise", "green1", 
               "yellow4","darkgreen","black", "thistle4"),
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "PCoA on Aitchison Distances") 

embed_notebook(p)

<hr>

### Non-Metric Multidimensional Scaling

In [ ]:
bac_aitchison_nmds_2D <- metaMDS(bac_aitchison_dist,k=2,trymax=20)
bac_aitchison_nmds_3D <- metaMDS(bac_aitchison_dist,k=3,trymax=20)

In [ ]:
cat("\n", "Stress 3D: ", bac_aitchison_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_aitchison_nmds_2D[["stress"]])

stressplot(bac_aitchison_nmds_3D)

In [ ]:
bac_aitchison_nmds_3D_plot_data <- as.data.frame(bac_aitchison_nmds_3D[["points"]])

bac_aitchison_nmds_3D_plot_data$sampleid <- rownames(bac_aitchison_nmds_3D_plot_data)

bac_aitchison_nmds_3D_plot_data <-
  left_join(bac_aitchison_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_aitchison_nmds_3D_plot_data,
    x      = ~MDS1,
    y      = ~MDS2,
    z      = ~MDS3,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = theme,
                      yaxis = theme,
                      zaxis = theme),
         title = "NMDS on Aitchison Distances")

embed_notebook(p)

<br><hr><br>
<a id='PhILRDistance'></a>

# PhILR Distance

[&#8593; back to content &#8593;](#Content)

The **Ph**ylogenetic **I**sometric **L**og-**R**atio transformation combines evolutionary models with the isometric log-ratio transform and projects the data into a cartesian space.

https://doi.org/10.7554/eLife.21887.001


In [ ]:
otu.file <- file.path("/root/biommar/data/B_asv.txt")
tax.file <- file.path("/root/biommar/data/B_asv_taxonomy.txt")
meta.file <- file.path("/root/biommar/data/B_asv_metadata.txt")

bac_ps <- read_phyloseq(
  otu.file = otu.file,
  taxonomy.file = "/root/biommar/data/taxa.txt",
  metadata.file = meta.file,
  type = "simple",
  sep = "\t"
)

meta <- read.table(meta.file, row.names = 1, header = T, sep="\t")

## import tree
tree <- read_tree("/root/biommar/data/tree.nwk")

bac_ps <- merge_phyloseq(bac_ps, tree)

tse <- mia::makeTreeSummarizedExperimentFromPhyloseq(bac_ps)

tse

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#
# Filter Extremely Low-Abundance OTUs
#
# Taxa that were not seen with more than 3 counts in at least 20% of samples are
# filtered. Subsequently, those with a coefficient of variation ≤ 3 are
# filtered. Finally we add a pseudocount of 1 to the remaining OTUs to avoid
# calculating log-ratios involving zeros.

## Select prevalent taxa 
tse <-  tse %>% subsetByPrevalentTaxa(
  detection = 0.1,
  prevalence = 0.001,
  as_relative = FALSE)

## Pick taxa that have notable abundance variation across sammples
variable.taxa <- apply(assays(tse)$counts, 1, function(x) sd(x)/mean(x) > 2.0)
tse <- tse[variable.taxa,]

tse

# Collapse the tree!
# Otherwise the original tree with all nodes is kept
# (including those that were filtered out from rowData)
tree <- ape::keep.tip(phy = rowTree(tse), tip = rowLinks(tse)$nodeNum)
rowTree(tse) <- tree

## Add a new assay with a pseudocount 
assays(tse)$counts.shifted <- assays(tse)$counts + 1e-12

tree <- makeNodeLabel(tree, method="number", prefix='n')

In [ ]:
is.rooted(tree)

In [ ]:
is.binary(tree)
# if FALSE
# tree <- ape::multi2di(tree)

In [ ]:
# Add the modified tree back to the (`TreeSE`) data object 
rowTree(tse) <- tree

# Extract taxonomy table from the TreeSE object
tax <- rowData(tse)[,taxonomyRanks(tse)]

# Get name balances
name.balance(tree, tax, 'n1')

In [ ]:
otu.table <- t(as(assays(tse)$counts.shifted, "matrix"))
tree <- rowTree(tse)
metadata <- colData(tse)
tax <- rowData(tse)[,taxonomyRanks(tse)]

data_philr <- philr(tse, 
                    part.weights = 'uniform',
                    ilr.weights  = 'uniform', 
                    return.all   = FALSE,
                    abund_values = "counts.shifted")

# Distances between samples based on philr transformed data
bac_philr_dist <- dist(data_philr, method = "euclidean") 

# compute the PCoA
bac_philr_pcoa <- ape::pcoa(data_dist)

# save PCo vector data for plot
bac_philr_pcoa_data <-
  data.frame(
    Sample = as.character(rownames(bac_philr_pcoa$vectors)),
    X = bac_philr_pcoa$vectors[, 1],
    Y = bac_philr_pcoa$vectors[, 2],
    Z = bac_philr_pcoa$vectors[, 3]
  )

# add metadata
bac_philr_pcoa_data$Sample <- rownames(bac_philr_pcoa_data)

colnames(bac_philr_pcoa_data) <- c("sampleid", "X", "Y", "Z")

bac_philr_pcoa_data <- left_join(bac_philr_pcoa_data, bac_metadata, by = "sampleid")
  
# plotting
axx <- list(title    = paste("PCo 1 - ", 
                             round(bac_philr_pcoa$values$Relative_eig[[1]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axy <- list(title    = paste("PCo 2 - ", 
                             round(bac_philr_pcoa$values$Relative_eig[[2]] * 100, 2),"%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")
axz <- list(title    = paste("PCo 3 - ", 
                             round(bac_philr_pcoa$values$Relative_eig[[3]] * 100, 2), "%", sep = ""),
            tickfont = list(color = c('#323232')), 
            color    = "#aaaaaa")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data   = bac_philr_pcoa_data,
    x      = ~X,
    y      = ~Y,
    z      = ~Z,
    color  = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(size = 5, opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis = axx,
                      yaxis = axy,
                      zaxis = axz),
         title = "\nPCoA on Euclidian Distances") 

embed_notebook(p)

In [ ]:
bac_philr_nmds_2D <- metaMDS(bac_philr_dist, k = 2, trymax = 20)
bac_philr_nmds_3D <- metaMDS(bac_philr_dist, k = 3, trymax = 20)

In [ ]:
cat("\n", "Stress 3D: ", bac_philr_nmds_3D[["stress"]], "\n",
    "Stress 2D: ", bac_philr_nmds_2D[["stress"]])

stressplot(bac_philr_nmds_3D)

In [ ]:
bac_philr_nmds_3D_plot_data <- as.data.frame(bac_philr_nmds_3D[["points"]])

bac_philr_nmds_3D_plot_data$sampleid <- rownames(bac_philr_nmds_3D_plot_data)

bac_philr_nmds_3D_plot_data <-
  left_join(bac_philr_nmds_3D_plot_data, bac_metadata, by = "sampleid")

p <- plot_ly(width = 900, height = 700) %>%
  add_trace(
    type='scatter3d',
    mode='markers',
    data = bac_philr_nmds_3D_plot_data,
    x = ~MDS1,
    y = ~MDS2,
    z = ~MDS3,
    color = ~department,
    colors = c("#00468B", "#ED0000", "#42B540", "#0099B4", "#925E9F"),
    marker = list(
      size = 5,
      opacity = 1.0),
    showlegend = T) %>% 
  layout(scene = list(xaxis=theme,yaxis=theme,zaxis=theme),
         title = "\nNMDS on Euclidian Distances")

embed_notebook(p)

<br><hr><br>
<a id='Statistics'></a>

# Statistics

[&#8593; back to content &#8593;](#Content)

### Pairwise ADONIS

In [ ]:
# https://github.com/pmartinezarbizu/pairwiseAdonis/blob/master/pairwiseAdonis/R/pairwise.adonis.R

pairwise.adonis <- function(x, factors, sim.function = 'vegdist', sim.method = 'bray', 
                            p.adjust.m ='bonferroni', reduce=NULL, perm=999){
  require("vegan")
  co <- combn(unique(as.character(factors)),2)
  pairs <- c()
  Df <- c()
  SumsOfSqs <- c()
  F.Model <- c()
  R2 <- c()
  p.value <- c()
  for(elem in 1:ncol(co)){
    if(inherits(x, 'dist')){
      x1=as.matrix(x)[factors %in% c(as.character(co[1,elem]), as.character(co[2,elem])),
                      factors %in% c(as.character(co[1,elem]), as.character(co[2,elem]))]
    }
    else  (
      if (sim.function == 'daisy'){
        x1 = daisy(x[factors %in% c(co[1,elem],co[2,elem]),],metric=sim.method)
      } 
      else{x1 = vegdist(x[factors %in% c(co[1,elem],co[2,elem]),], method=sim.method)}
    )
    x2 = data.frame(Fac = factors[factors %in% c(co[1,elem],co[2,elem])])
    ad <- adonis2(x1 ~ Fac, data = x2, permutations = perm);
    pairs <- c(pairs,paste(co[1,elem],'-',co[2,elem]));
    Df <- c(Df,ad$Df[1])
    SumsOfSqs <- c(SumsOfSqs,ad$SumOfSqs[1])
    F.Model <- c(F.Model,ad$F[1]);
    R2 <- c(R2,ad$R2[1]);
    p.value <- c(p.value,ad$`Pr(>F)`[1])
  }
  p.adjusted <- p.adjust(p.value,method=p.adjust.m)
  sig = c(rep('',length(p.adjusted)))
  sig[p.adjusted <= 0.1] <-'.'
  sig[p.adjusted <= 0.05] <-'* '
  sig[p.adjusted <= 0.01] <-'**'
  sig[p.adjusted <= 0.001] <-'***'
  pairw.res <- data.frame(pairs,Df,SumsOfSqs,F.Model,R2,p.value,p.adjusted,sig)
  if(!is.null(reduce)){
    pairw.res <- subset (pairw.res, grepl(reduce,pairs))
    pairw.res$p.adjusted <- p.adjust(pairw.res$p.value,method=p.adjust.m)
    sig = c(rep('',length(pairw.res$p.adjusted)))
    sig[pairw.res$p.adjusted <= 0.1] <-'.'
    sig[pairw.res$p.adjusted <= 0.05] <-'* '
    sig[pairw.res$p.adjusted <= 0.01] <-'**'
    sig[pairw.res$p.adjusted <= 0.001] <-'***'
    pairw.res <- data.frame(pairw.res[,1:7],sig)
  }
  class(pairw.res) <- c("pwadonis", "data.frame")
  return(pairw.res)
} 


In [ ]:
philr.df <- as.data.frame(data_philr)
meta <- read.table(meta.file, row.names = 1, header = T, sep="\t")
#meta$country[is.na(meta$country)] <- "Curazao"

philr.dist <- vegdist(philr.df, "euclidean")

disp.country <- permutest(betadisper(philr.dist, meta$country), 
                          pairwise = TRUE, permutations = how(nperm = 999))

disp.Clade <- permutest(betadisper(philr.dist, meta$Clade), 
                        pairwise = TRUE, permutations = how(nperm = 999))

disp.locality <- permutest(betadisper(philr.dist, meta$locality), 
                           pairwise = TRUE, permutations = how(nperm = 999))

disp.department <- permutest(betadisper(philr.dist, meta$department), 
                           pairwise = TRUE, permutations = how(nperm = 999))

philr.dist <- as.data.frame(as.matrix(vegdist(philr.df, "euclidean")))

x1 <- capture.output(print(disp.country), file = NULL)
x2 <- capture.output(print(disp.Clade), file = NULL)
x3 <- capture.output(print(disp.locality), file = NULL)
x4 <- capture.output(print(disp.department), file = NULL)

gsub(" \\(t\\)", "" , names(disp.country[["statistic"]][2:length(disp.country[["statistic"]])]))

#paste(x, collapse = '\n') %>% cat()

pwa.country <- pairwise.adonis(data_philr , meta$country, 
                               p.adjust.m = "bonferroni",
                               sim.method = "euclidean", perm = 999)

pwa.Clade <- pairwise.adonis(data_philr , meta$Clade, 
                             p.adjust.m = "bonferroni",
                             sim.method = "euclidean", perm = 999)

pwa.locality <- pairwise.adonis(data_philr , meta$locality, 
                                p.adjust.m = "bonferroni",
                                sim.method = "euclidean", perm = 999)

pwa.department <- pairwise.adonis(data_philr , meta$department, 
                                  p.adjust.m = "bonferroni",
                                  sim.method = "euclidean", perm = 999)

rownames(pwa.country) <- names(disp.country[["pairwise"]][["permuted"]])


pwa.country$disp <- disp.country[["pairwise"]][["permuted"]]
pwa.country$disp.sig <- 
  ifelse (pwa.country$disp <= 0.001, "***",
          ifelse (pwa.country$disp <= 0.01, "**",
                  ifelse (pwa.country$disp <= 0.05, "*",
                          ifelse(pwa.country$disp <= 0.1, ".", ""))))

pwa.Clade$disp <- disp.Clade[["pairwise"]][["permuted"]]
pwa.Clade$disp.sig <- 
  ifelse (pwa.Clade$disp <= 0.001, "***",
          ifelse (pwa.Clade$disp <= 0.01, "**",
                  ifelse (pwa.Clade$disp <= 0.05, "*",
                          ifelse(pwa.Clade$disp <= 0.1, ".", ""))))

pwa.locality$disp <- disp.locality[["pairwise"]][["permuted"]]
pwa.locality$disp.sig <- 
  ifelse (pwa.locality$disp <= 0.001, "***",
          ifelse (pwa.locality$disp <= 0.01, "**",
                  ifelse (pwa.locality$disp <= 0.05, "*",
                          ifelse(pwa.locality$disp <= 0.1, ".", ""))))


pwa.department$disp <- disp.department[["pairwise"]][["permuted"]]
pwa.department$disp.sig <- 
  ifelse (pwa.department$disp <= 0.001, "***",
          ifelse (pwa.department$disp <= 0.01, "**",
                  ifelse (pwa.department$disp <= 0.05, "*",
                          ifelse(pwa.department$disp <= 0.1, ".", ""))))

In [ ]:
pwa.country %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwa.Clade %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwa.locality %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwa.department %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

### Pairwise ANOSIM

In [ ]:
pairwise_anosim <- function(x, meta_data, group, 
                            permutations = 999, distance = "bray")
{
  .group <- unique(meta_data[, group])
  .combs <- combn(.group, 2)
  .res <- data.frame(matrix(ncol = 4, nrow = length(.combs)/2))
  require("vegan")
  cat(paste("Pairwise ANOSIM of", length(.combs)/2, "combinations.\n   
              Permutations: ", permutations))
  for(i in seq(length(.combs)/2)) {
    .this_data <- x[meta_data[, group] %in% .combs[,i], ]
    .this_meta <- meta_data[meta_data[, group] %in% .combs[,i], ]
    .tmp <- anosim(.this_data, .this_meta[, group], 
                   permutations = permutations, distance = distance)
    .var <- as.character(unique(.tmp[["class.vec"]]))
    .sig <- as.character(.tmp[["signif"]])
    .R <- as.character(.tmp[["statistic"]])
    .res[i,] <- c(.var[-which(.var=="Between")], .sig, .R) 
    cat(paste("\r    ",i, "of", length(.combs)/2, "done"), fill = TRUE)
  }
  colnames(.res) <- c("variable.1", "variable.2", "p.value", "R.value")
  .res$p.value <- as.numeric(.res$p.value)
  .res <- .res[order(.res$p.value), ]
  .res$adj.p <- .res$p.value * rev(seq(1:nrow(.res)))
  .dis <- vegdist(x, method = distance)
  .bet <- permutest(betadisper(.dis, meta_data[,group]), pairwise = TRUE)
  .p <- .bet[["tab"]][["Pr(>F)"]][1]
  .res$dispersion.p <- .p
  return(.res)
}

In [ ]:
pwn.country <- pairwise_anosim(data_philr , meta, "country", 
                               distance = "euclidean",
                               permutations = 999)


In [ ]:
pwn.country %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwn.department <- pairwise_anosim(data_philr , meta, "department", 
                                  distance = "euclidean", 
                                  permutations = 999)


In [ ]:
pwn.department %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwn.Clade <- pairwise_anosim(data_philr , meta, "Clade", 
                                  distance = "euclidean", 
                                  permutations = 999)


In [ ]:
pwn.Clade %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
pwn.morpho_specie <- pairwise_anosim(data_philr , meta, "morpho_specie", 
                                  distance = "euclidean", 
                                  permutations = 999)


In [ ]:
pwn.morpho_specie %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

<hr>

In [ ]:
pairwise_anosim_plot <- function(x, meta_data, group, 
                                 permutations = 999, distance = "bray")
{
  require("ggplot2")
  require("reshape")
  require("vegan")
  require("multcompView")
  require("dplyr")
  .pdat <- pairwise_anosim(x, meta_data, group, 
                           permutations = permutations, distance = distance)
  .beta <- unique(.pdat[,ncol(.pdat)])
  .pdat <- .pdat[,-ncol(.pdat)]
  .tmp <- anosim(x, meta_data[, group], permutations = permutations, distance = distance)
  # extract ranks to plot on y axis and respective variable to plot on x axis
  .xdat <- data.frame(t(as.data.frame(rbind(
    as.character(.tmp[["class.vec"]]), .tmp[["dis.rank"]]))))
  .xdat$X2 <- as.numeric(.xdat$X2)
  # make symmetric matrix of P-values for compact letter display
  .x1 <- reshape::cast(.pdat, variable.1 ~ variable.2, value = "adj.p", fill = 0)
  .x2 <- reshape::cast(.pdat, variable.2 ~ variable.1, value = "adj.p", fill = 0)
  rownames(.x1) <- .x1$variable.1 
  rownames(.x2) <- .x2$variable.2 
  # casting produces X*X-1 matrix; solve by merging X*X-1 and X-1*X matrices
  .z1 <- base::merge(.x1, .x2, by = "row.names", all = T)
  rownames(.z1) <- .z1$Row.names
  .z1 <- .z1[, 2:ncol(.x1) + 1]
  .z2 <- merge(.x2, .x1, by ="row.names", all = T)
  rownames(.z2) <- .z2$Row.names
  .z2 <- .z2[, 2:ncol(.x1) + 1]
  .z <- merge(.z1, .z2, by = "row.names", all = T)
  rownames(.z) <- .z$Row.names
  .z <- .z[ , -1]
  .z <- t(.z)
  # get rid of duplicate names
  .out <- .z[grep(".x.y", row.names(.z)), ]
  .out <- rownames(.out)
  .z <- subset(.z, !(rownames(.z) %in% .out))
  .z <- as.data.frame(.z)
  rownames(.z) <- gsub(".x.x$", "", rownames(.z))
  .z <- .z[order(rownames(.z)), ]
  .z <- .z[order(colnames(.z)), ]
  diag(.z) <- 1
  .z[.z == 0] <- NA
  # fill empty cells with diagonal pair
  for (i in 1:ncol(.z)){
    for (j in 1:nrow(.z)){
      if (!is.na(.z[j,i])){
        .z[i,j] <- .z[j,i]
      }
    }
  }
  .let <- multcompView::multcompLetters(.z)
  .let <- as.data.frame(.let[["Letters"]])
  .let$X1 <- rownames(.let)
  .plt <- merge(.xdat, .let, by ="X1", all = T)
  .p <- ggplot(.plt, aes(x = X1, y = X2, fill = X1)) + 
    geom_jitter(aes(color = X1), alpha = 0.2, size = 2) +
    geom_boxplot(position = position_dodge2(),
                 alpha = 0.9, varwidth = TRUE) +  
    geom_label(data = .plt %>% 
               group_by(X1) %>% 
               summarise(y = max(X2), l = `.let[["Letters"]]`),
               aes(y = y, label = l), position = position_dodge(width = 1), 
               size = 5, stat = "unique", parse = TRUE, vjust = -.8) +
    theme_bw() +
    theme(legend.position ="none",
          plot.title = element_text(size = 25, hjust = 0.5, face = "bold", 
                                    color = "black"),
          plot.subtitle = element_text(size = 18, hjust = 0.5, face = "italic", 
                                       color = "gray15"),
          text = element_text(size = 18),
          axis.text = element_text(size = 12, face ="bold"),
          axis.title = element_text(size = 18, face = "bold"),
          plot.caption = element_text(size = 10))+
    xlab(paste("\n",group)) +
    ylab("Rank\n") +
    labs(title = "Pairwise ANOSIM",
         subtitle = "Holm-Bonferroni correction of P-values for \
         compact letter display\n",
         caption = paste("Beta Dispersion", .beta, 
                         "\tPermutations:", permutations)) +
    expand_limits(y = c(0, max(.plt$X2)*1.1))
  return(.p)
}

In [ ]:
plt_specie <- pairwise_anosim_plot(data_philr, 
                                   bac_metadata, 
                                   "morpho_specie",
                                   distance = "euclidian",
                                   permutations = 999) + 
  scale_shape_manual(values=rep(c(21,22,23,24,25), 5)) 
#+
  #scale_colour_manual(values = 
   #                     c("dodgerblue2", "#E31A1C", "green4", "#6A3D9A", 
    #                      "#FF7F00", "gold1", "maroon", "orchid1", "deeppink1", 
     #                     "blue1", "steelblue4", "darkturquoise", "green1", 
      #                    "yellow4","green","blue")) +
  #scale_fill_manual(values = 
   #                   c("dodgerblue2", "#E31A1C", "green4", "#6A3D9A", 
    #                    "#FF7F00", "gold1", "maroon", "orchid1", "deeppink1", 
     #                   "blue1", "steelblue4", "darkturquoise", "green1", 
      #                  "yellow4","green","blue"))
    
plt_specie

In [ ]:
plt_department <- pairwise_anosim_plot(data_philr, 
                                       bac_metadata, 
                                       "department",
                                       distance = "euclidian",
                                       permutations = 999) + 
  scale_fill_lancet() + 
  scale_color_lancet()

plt_department

In [ ]:
plt_clade <- pairwise_anosim_plot(data_philr, 
                                  bac_metadata, 
                                  "Clade",
                                  distance = "euclidian",
                                  permutations = 999) +
  scale_shape_manual(values=rep(c(21,22,23,24,25), 5)) + 
  theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1))

plt_clade

In [ ]:
res.dep <- pairwise.adonis(bac_feature_table_css, 
                           bac_metadata_css$department, 
                           perm = 999)

In [ ]:
res.dep %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
res.dep <- pairwise.adonis(bac_feature_table_css, 
                           bac_metadata_css$department, 
                           perm = 999)

In [ ]:
res.dep %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

In [ ]:
res.dep <- pairwise.adonis(bac_feature_table_css, 
                           bac_metadata_css$department, 
                           perm = 999)

In [ ]:
res.dep %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:100%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  kable_styling(c("striped", "bordered"), full_width = F, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()

<br><hr><br>

# t-SNE

[&#8593; back to content &#8593;](#Content)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4.5, repr.plot.res = 300)

In [ ]:
tsne_indata <- merge(bac_feature_table, bac_metadata, by = 'row.names')
rownames(tsne_indata) <- tsne_indata$Row.names
tsne_indata <- tsne_indata[,-1]

bac_tsne_dat <- Rtsne::Rtsne(tsne_indata[, 1:(ncol(tsne_indata)-12)])

# extract data
tsne_plot <- data.frame(x = bac_tsne_dat$Y[, 1],
                        y = bac_tsne_dat$Y[, 2],
                        tsne_indata[, (ncol(tsne_indata)-11):ncol(tsne_indata)])

  ggplot(tsne_plot) + 
  geom_point(aes(x = x, y = y, color = department, size = depth)) +
  scale_color_lancet() +
  theme_bw() +
  theme(
    # text = element_text(size = 18),
    # axis.text = element_text(size = 12),
    # axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(hjust = 0.5)
  ) +
  ggtitle("T-Distributed Stochastic Neighbor Embedding \n") +
  xlab("") +
  ylab("")

In [ ]:
tsne_indata <- merge(bac_feature_table_css, bac_metadata_css, by = 'row.names')
rownames(tsne_indata) <- tsne_indata$Row.names
tsne_indata <- tsne_indata[,-1]

bac_tsne_dat <- Rtsne::Rtsne(tsne_indata[, 1:(ncol(tsne_indata)-12)])

# extract data
tsne_plot <- data.frame(x = bac_tsne_dat$Y[, 1],
                        y = bac_tsne_dat$Y[, 2],
                        tsne_indata[, (ncol(tsne_indata)-11):ncol(tsne_indata)])

  ggplot(tsne_plot) + 
  geom_point(aes(x = x, y = y, color = department, size = depth)) +
  scale_color_lancet() +
  theme_bw() +
  theme(
    # text = element_text(size = 18),
    # axis.text = element_text(size = 12),
    # axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(hjust = 0.5)
  ) +
  ggtitle("T-Distributed Stochastic Neighbor Embedding (CSS)\n") +
  xlab("") +
  ylab("")

In [ ]:
# prepare data
A.tsne_indata <- as.matrix(tsne_indata[, 1:(ncol(tsne_indata)-12)])
# add pseudo count 
A.tsne_indata[A.tsne_indata == 0] <- 1e-12
# transform to Aitchison distance
A.vars <- t(apply(A.tsne_indata, 1, function(x)log(x/exp(mean(log(x))))))
A.dist <- as.matrix(dist(A.vars))

# compute t-SNE with arbitrary parameters
bac_tsne_dat <- Rtsne(A.dist, 
                      initial_dims = 5, # manipulate
                      perplexity = 25,   # "
                      max_iter = 50000,   # "
                      is_distance = T)

# extract data for plotting
tsne_plot <- data.frame(x = bac_tsne_dat$Y[, 1],
                        y = bac_tsne_dat$Y[, 2],
                        tsne_indata[, (ncol(tsne_indata)-11):ncol(tsne_indata)])

  ggplot(tsne_plot) + 
  geom_point(aes(x = x, y = y, color = department, size = depth)) +
  scale_color_lancet() +
  theme_bw() +
  theme(
    #text = element_text(size = 18),
    #axis.text = element_text(size = 12),
    #axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(hjust = 0.5, size = 16)
  ) +
  ggtitle("T-Distributed Stochastic Neighbor Embedding (Aitchison Distance)\n") +
  xlab("") +
  ylab("")

In [ ]:
ggplot(tsne_plot) + 
  geom_point(aes(x = x, y = y, color = Clade, 
                 fill = Clade, shape = Clade), size = 3) +
  scale_shape_manual(values=rep(c(21,22,23,24,25), 5)) +
  scale_colour_manual(values = 
                        c("dodgerblue2", "#E31A1C", "green4", "#6A3D9A", 
                          "#FF7F00", "gold1", "maroon", "orchid1", "deeppink1", 
                          "blue1", "steelblue4", "darkturquoise", "green1", 
                          "yellow4","darkgreen", "black", "thistle4")) +
  scale_fill_manual(values = 
                      c("dodgerblue2", "#E31A1C", "green4", "#6A3D9A", 
                        "#FF7F00", "gold1", "maroon", "orchid1", "deeppink1", 
                        "blue1", "steelblue4", "darkturquoise", "green1", 
                        "yellow4","darkgreen","black", "thistle4")) +
  theme_bw() +
  theme(
    #text = element_text(size = 18),
    #axis.text = element_text(size = 12),
    #axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(hjust = 0.5, size = 18)
  ) +
  ggtitle("T-Distributed Stochastic Neighbor Embedding (Aitchison Distance)\n") +
  xlab("") +
  ylab("")

<br><hr><br>

# Composition

[&#8593; back to content &#8593;](#Content)

In [ ]:
to_incidence_freq <- function(feature_table=bac_feature_table, 
                              meta_data=bac_metadata, var_group){
  
  group <- deparse(substitute(var_group))
  a <- as.data.frame(ifelse(feature_table > 0, 1, 0))
  a$sampleid <- rownames(a)
  b <- dplyr::left_join(a, meta_data[,c("sampleid", group)], by = "sampleid")
  b <- b[,c(1:(ncol(b)-2),ncol(b))]
  c <- b %>% dplyr::group_by({{ var_group }}) %>% dplyr::summarise(dplyr::across(dplyr::everything(), sum))
  class(c) <- "data.frame"
  d <- as.list(as.data.frame(t(c)))
  names(d) <- base::unique(meta_data[,group])
  d <- lapply(d, function(x) 
    as.numeric(c(length(b[,group][b[,group] == x[1]]), x[2:length(x)])))
  d <- lapply(d, function(x) {x[x!=0]})
  return(d)
}

<br><hr><br>

<h2>Coverage - Clade</h2>
<br>

In [ ]:
next_data_Cla <- to_incidence_freq(var_group=Clade)

next_res_Cla <- iNEXT(next_data_Cla, q=c(0,1,2), datatype="incidence_freq")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 200)

In [ ]:
plot_next_cov_Cla <- ggiNEXT(next_res_Cla, type=2, se=TRUE, facet.var="site", 
                         color.var="site", grey=TRUE) +
  theme_bw() +
  theme(legend.position = "none")
  
plot_next_cov_Cla

Sample coverage is in most cases high enough for reliable analysis, i.e. coverage of >= 50% (Chao and Jost 2012).



<hr><br>

<h2>Richness - Clade</h2>
<br>

In [ ]:
plot_next_rich_Cla <- ggiNEXT(next_res_Cla, type=1, se=TRUE, facet.var="site", 
                           color.var="site") +
  scale_color_lancet() +
  scale_fill_lancet() +
  theme_bw() +
  theme(legend.position = "bottom")

plot_next_rich_Cla[["guides"]][["colour"]][["title"]] <- "Hill Number"
plot_next_rich_Cla[["guides"]][["fill"]][["title"]] <- "Hill Number"
plot_next_rich_Cla[["guides"]][["shape"]][["title"]] <- "Hill Number"

plot_next_rich_Cla

<hr><br>

<h2>Coverage - Department</h2>
<br>

In [ ]:
next_data_dep <- to_incidence_freq(var_group=department)

next_res_dep <- iNEXT(next_data_dep, q=c(0,1,2), datatype="incidence_freq")


plot_next_cov_dep <- ggiNEXT(next_res_dep, type=2, se=TRUE, facet.var="none", 
                             color.var="site") +
  scale_color_lancet() +
  scale_fill_lancet()

data1 <- rbind(t(plot_next_cov_dep[["layers"]][[1]][["data"]][["x"]]),
               t(plot_next_cov_dep[["layers"]][[1]][["data"]][["y"]])) %>% 
  as.matrix() %>% 
  t() %>% 
  as.data.frame() %>% 
  "colnames<-"(c("x","y")) %>% 
  mutate(department = next_res_dep[["DataInfo"]][["site"]])

In [ ]:
plot_next_cov_dep <- 
  plot_next_cov_dep + 
  geom_point(data = data1, aes(color = department, fill = department,
                              shape = department), size =6) +
  scale_shape_manual(values=c(21:25)) +
  theme_bw()

plot_next_cov_dep

<hr><br>

<h2>Richness - Department</h2>
<br>

In [ ]:
plot_next_rich_dep <- ggiNEXT(next_res_dep, type=1, se=TRUE, facet.var="order", 
                          color.var="site") +
  scale_color_lancet() +
  scale_fill_lancet()


data2 <- rbind(t(plot_next_rich_dep[["layers"]][[1]][["data"]][["x"]]),
               t(plot_next_rich_dep[["layers"]][[1]][["data"]][["y"]])) %>% 
  as.matrix() %>% 
  t() %>% 
  as.data.frame() %>% 
  "colnames<-"(c("x","y")) %>% 
  mutate(department = rep(next_res_dep[["DataInfo"]][["site"]],each = 3)) %>% 
  mutate(order = rep(0:2,5))

plot_next_rich_dep <- 
  plot_next_rich_dep + 
  geom_point(data = data2, aes(color = department, fill = department,
                              shape = department), size = 6) +
  scale_shape_manual(values=c(21:25)) +
  theme_bw()

plot_next_rich_dep 

<hr><br>

<h2>Evenness</h2>

In [ ]:
dat_even <- as.data.frame(evenness(bac_ps))

dat_even <- merge(dat_even, bac_metadata_css, by = "row.names")

plot_evenness <-
  ggplot(dat_even, aes(x=department, y=pielou, color=department, 
                       fill=department)) + 
  geom_violin(trim = FALSE, alpha = 0.6) +
  geom_boxplot(width = 0.1, color = "black", outlier.color = "black",
               outlier.alpha = 0.5, outlier.size = 2.5) +
  theme_bw() +
  scale_fill_lancet() +
  scale_color_lancet() + 
  xlab("\ndepartment") +
  ylab("Pielou's Evenness Index\n") +
  theme(
    text = element_text(size = 14),
    axis.text = element_text(size = 10),
    axis.text.x = element_text(angle = -20),
    axis.title = element_text(size = 12, face = "bold"),
    plot.subtitle = element_text(vjust = 0.5, hjust = 0.5, size = 10),
    plot.title = element_text(hjust = 0.5, size = 18),
    legend.position = "none")


# bf.test(pielou ~ department, data = dat_even)

# hist(dat_even$pielou)

# shapiro.test(dat_even$pielou)

# oneway.test(pielou ~ department, data = dat_even)

# kruskal.test(pielou ~ department, data = dat_even)

even_aov <- aov(pielou ~ department, data = dat_even)

# plot(even_aov)

aov_test <- summary(even_aov)

even_aov_post <- TukeyHSD(even_aov)

plot_evenness <- 
  plot_evenness + labs(subtitle = 
                         paste("AOV test:",
                               "  F value = ", 
                               signif(aov_test[[1]]$`F value`[1], 4), 
                               ",  Df = ",
                               aov_test[[1]]$Df[1], 
                               ",  Pr(>F) = ", 
                               signif(aov_test[[1]]$`Pr(>F)`[1], 2), 
                               "\n", sep = "", collapse = ""),
                       title = "Evenness\n") +
  geom_segment(aes(x=3,y=1.25,xend=5,yend=1.25), color="black") +
  annotate("text", x = 4, y = 1.3, label = "**") +
  ggtitle("")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6, repr.plot.res = 200)

In [ ]:
plot_evenness

<hr><br>

<h2>Clade X Department</h2>
<br>

In [ ]:
x <- as.data.frame(cbind(bac_metadata$Clade, bac_metadata$department))
x$V3 = 1
y <- cast(x, V1~V2, value="V3")
y %>% 
  mutate(`Bocas del Toro` = 
           cell_spec(`Bocas del Toro`, background = 
                       ifelse(`Bocas del Toro` != 0, "white","#ff8178"))) %>% 
  mutate(`Cartagena` = 
           cell_spec(`Cartagena`, background = 
                       ifelse(`Cartagena` != 0,"white","#ff8178"))) %>% 
  mutate(`Curazao` = 
           cell_spec(`Curazao`, background = 
                       ifelse(`Curazao` != 0, "white","#ff8178"))) %>% 
  mutate(`Long Keys Florida` = 
           cell_spec(`Long Keys Florida`, background = 
                       ifelse(`Long Keys Florida` != 0, "white","#ff8178"))) %>% 
  mutate(`San Andres Isla` = 
           cell_spec(`San Andres Isla`, background = 
                       ifelse(`San Andres Isla` != 0, "white","#ff8178"))) %>% 
  kable(align = "c", format = "html", 
        table.attr = "style='width:60%;'", escape = FALSE) %>%
  column_spec(1, bold = T) %>% 
  column_spec(c(1:ncol(y)), width = "18em") %>% 
  kable_styling(c("striped", "bordered"), full_width = T, font_size = 14, 
                position = "center") %>%
as.character() %>%
display_html()


<br><hr><br>

# iNEXT

[&#8593; back to content &#8593;](#Content)


<h2>Point Estimate (50% Coverage)</h2>
<br>

In [ ]:
next_50_cov_Cla <- estimateD(next_data_Cla, datatype="incidence_freq",
          base="coverage", level=0.5, conf=0.95)

In [ ]:
ggplot(next_50_cov_Cla, aes(x=as.factor(order), y=qD, fill=site, ymin=qD.LCL, 
                            ymax=qD.UCL)) +
  geom_col(stat="identity", position ="dodge2") +
  facet_grid(~as.factor(order), scale="free_x") +
  geom_errorbar(stat="identity", position ="dodge2") +
  geom_text(aes(label=site, y=15), 
            position = position_dodge(width = 0.9), angle = 90, hjust=0.05) +
  theme_bw() +
  theme(legend.position = "none") +
  scale_fill_grey() + xlab("") + ylab("")

<hr><br>

<h2>Point Estimate (90% Coverage)</h2>
<br>


In [ ]:
next_90_cov_dep <- estimateD(next_data_dep, datatype="incidence_freq",
                         base="coverage", level=0.9, conf=0.95)

In [ ]:
ggplot(next_90_cov_dep, aes(x=as.factor(order), y=qD, fill=site, ymin=qD.LCL, 
                            ymax=qD.UCL)) +
  geom_col(stat="identity", position ="dodge2") +
  facet_wrap(~as.factor(order), scale="free_x") +
  geom_errorbar(stat="identity", position ="dodge2") +
  theme_bw() +
  scale_fill_grey() + xlab("") + ylab("")

<br><hr><br>
<a id='CoreMicrobiome'></a>

# Core Microbiome

[&#8593; back to content &#8593;](#Content)

In [ ]:
taxa <- read.table("/root/biommar/data/taxa.txt")

In [ ]:
# make data relative
bac_feature_table_relative <- as.data.frame(apply(
  bac_feature_table, 1, function(x) { x / sum(x)}
))

# add metadata
bac_feature_table_relative_taxa <- merge(bac_feature_table_relative, 
                                         taxa, by = "row.names") 

rownames(bac_feature_table_relative_taxa) <- 
  bac_feature_table_relative_taxa$Row.names

bac_feature_table_relative_taxa  <- bac_feature_table_relative_taxa[,-1]


# filter Mycoplasmatales and Oceanospirillales
bac_sub_tax <-
  bac_feature_table_relative_taxa[bac_feature_table_relative_taxa$order %in%
                                    c("Mycoplasmatales","Oceanospirillales"), ]

# merge data based on order (exclude last 8 columns which are taxa info)
abund <- 
  cbind(
  colSums(bac_sub_tax[bac_sub_tax$order %in% 
                        c("Mycoplasmatales"), c(1:(ncol(bac_sub_tax)-8))]),
  colSums(bac_sub_tax[bac_sub_tax$order %in% 
                        c("Oceanospirillales"), c(1:(ncol(bac_sub_tax)-8))])
  )

colnames(abund) <- c("Mycoplasmatales","Oceanospirillales")

abund <- as.data.frame(abund)


ggplot(abund) +
  geom_density(aes(x = Mycoplasmatales, fill = "Mycoplasmatales"),
               color = "gray10", alpha = 0.5) +
  geom_density(aes(x = Oceanospirillales, fill = "Oceanospirillales"),
               color = "gray10", alpha = 0.5) +
  labs(x = "Relative abundance", title = "") +
  theme_classic() + # Changes the background
  scale_x_continuous(label = scales::percent) +
  scale_fill_lancet() +
  guides(fill=guide_legend(title="Order")) +
  theme(
    text = element_text(size = 18),
    axis.text = element_text(size = 12),
    axis.text.x = element_text(angle = -20),
    axis.title = element_text(size = 14, face = "bold"),
    plot.subtitle = element_text(vjust = 1.5, hjust = 0.85, size = 12),
    plot.title = element_text(hjust = 0.5, size = 24))


In [ ]:
# remove zero counts
bac_ps <- prune_taxa(taxa_sums(bac_ps) > 0, bac_ps)

# In transformation typ, the 'compositional' abundances are returned as
# relative abundances in [0, 1]
bac_ps_comp <- microbiome::transform(bac_ps, "compositional")


plot_heatmap(bac_ps_comp, method = "NMDS", distance = "jaccard") +
  geom_raster(aes(fill = log10(Abundance))) + 
  scale_fill_viridis_c(option="magma", 
                       na.value = "gray90", 
                       name = "log10(Abundance)",
                       direction = -1) +
  theme(axis.text.y = element_text(size=5))

In [ ]:
# define core microbiome based on detection and prevalence
bac_core_taxa <- core_members(bac_ps_comp,
                              detection = 0.001,
                              prevalence = 0.1)

bac_core_taxa <- as.data.frame(bac_core_taxa)
colnames(bac_core_taxa) <- c("sampleid")

write.table(bac_core_taxa,
            file = "/root/biommar/data/bac_core_taxa.tsv",
            sep = "\t",
            quote = F,
            row.names = F)

# add metadata
colnames(bac_core_taxa) <- c("ID")
rownames(bac_core_taxa) <- bac_core_taxa$ID
bac_core_taxa <- merge(bac_core_taxa, taxa, by = "row.names")

prevalences <- seq(.1, .5, .05)
detections <- round(10 ^ seq(log10(1e-3), log10(.2), length = 10), 5)

plot_bac_ps_core <- 
  plot_core(
    bac_ps_comp,
    plot.type = "heatmap",
    colours = gray,
    prevalences = prevalences,
    detections = detections,
    min.prevalence = .1) +
  xlab("\nDetection Threshold (Relative Abundance (%))") +
  theme_bw() +
  #ggtitle(paste("Identification Level :", filter_tax)) +
  ylab("ASVs") +
  scale_fill_viridis_c()

# replace ID by taxonomic name
names <- 
  plyr:::mapvalues(plot_bac_ps_core[["data"]][["Taxa"]], 
                   from = as.character(bac_core_taxa$ID), 
                   to = as.character(bac_core_taxa$genus))

plot_bac_ps_core + scale_y_discrete(labels = names) +
  theme(axis.text.y = element_text(size = 6),
        panel.border = element_blank(),
        panel.grid = element_blank())

<br><hr><br>
### Save R Environment

Saving the objects in the R environment to a file lets you access them faster after restarting the notebook. 

In [ ]:
save.image("/root/biommar/data/environment.RData")

<br><hr><br><br><hr><br>

# Interlude in bash

<span style="color:red">**PLEASE CHANGE KERNEL ACCORDINGLY!**</span>

Select ```Kernel``` in the menu bar at the top, click on ```Change kernel```, and select ```Python 3 (ipykernel)```.

<br><hr><br><br><hr><br>

In [ ]:
%%bash

source activate qiime2-2022.2

qiime feature-table filter-seqs \
    --i-data /root/biommar/data/dada_filtered_rep_seqs.qza \
    --m-metadata-file /root/biommar/data/bac_core_taxa.tsv \
    --o-filtered-data /root/biommar/data/core_sequences.qza

In [ ]:
%%bash

source activate qiime2-2022.2

qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences /root/biommar/data/core_sequences.qza \
    --p-n-threads 4 \
    --o-alignment /root/biommar/data/core_alignemnt.qza \
    --o-masked-alignment /root/biommar/data/core_masked_alignment.qza \
    --o-tree /root/biommar/data/core_unrooted_tree.qza \
    --o-rooted-tree /root/biommar/data/core_rooted_tree.qza

cd  /root/biommar/data/
unzip -qq core_rooted_tree.qza
cp $(echo $(ls -td -- /root/biommar/data/*/ | head -n1) $(echo "data/tree.nwk") | sed 's/\ //g') \
   /root/biommar/data/core_tree.nwk
rm -r $(ls -td -- /root/biommar/data/* | head -n 1)

<br><hr><br><br><hr><br>

# Continue in R

<span style="color:red">**PLEASE CHANGE KERNEL ACCORDINGLY!**</span>

Select ```Kernel``` in the menu bar at the top, click on ```Change kernel```, and select ```R```.

<br><hr><br><br><hr><br>

In [ ]:
library( ALDEx2,        quietly = TRUE)
library( ape,           quietly = TRUE)
library( aplot,         quietly = TRUE)
library( cluster,       quietly = TRUE)
library( ggdendro,      quietly = TRUE)
library( grid,          quietly = TRUE)
library( glmnet,        quietly = TRUE)
library( ggplot2,       quietly = TRUE)
library( ggfortify,     quietly = TRUE)
library( ggthemes,      quietly = TRUE)
library( gridExtra,     quietly = TRUE)
library( ggpmisc,       quietly = TRUE)
library( ggsci,         quietly = TRUE)
library( ggtree,        quietly = TRUE)
library( tidyr,         quietly = TRUE)
library( metagenomeSeq, quietly = TRUE)
library( microbiome,    quietly = TRUE)
library( phyloseq,      quietly = TRUE)
library( philr,         quietly = TRUE)
library( mia,           quietly = TRUE)
library( vegan,         quietly = TRUE)
library( multcompView,  quietly = TRUE)
library( plyr,          quietly = TRUE)
library( dplyr,         quietly = TRUE)
library( Rtsne,         quietly = TRUE)
library( plotly,        quietly = TRUE)
library( zgtools,       quietly = TRUE)
library( reshape,       quietly = TRUE)
library( onewaytests,   quietly = TRUE)
library( iNEXT,         quietly = TRUE)
library( phytools,      quietly = TRUE)
library( reshape2,      quietly = TRUE)
library( repr,          quietly = TRUE)
library( kableExtra,    quietly = TRUE)
library( IRdisplay,     quietly = TRUE)
library( jsonlite,     quietly = TRUE)

<br><hr><br>
### Load R Environment

Load the environment to restore objects. 

In [ ]:
load("/root/biommar/data/environment.RData")

<br><hr><br>
<a id='HostPhylogeny'></a>

# Host Phylogeny

[&#8593; back to content &#8593;](#Content)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 12, repr.plot.res = 200)

### Eunicea phylogeny plot 

In [ ]:
# read in the tree produced in Qiime2
host_tree <- read_tree("/root/biommar/data/subd_params-data4.phy.treefile")

host_tree[["tip.label"]] <- as.character(host_tree[["tip.label"]])

# get an overview of the phylogeny

ggtree(host_tree) + 
  geom_tiplab(size=2, align=TRUE, linesize=.5) +
  ggplot2::xlim(0, 0.05)

# crop off prefix and suffix added to ID during construction
host_tree[["tip.label"]] <- gsub("eunicea_", "", host_tree[["tip.label"]]) 
host_tree[["tip.label"]] <- gsub("_ctadpt1", "", host_tree[["tip.label"]])

# match metadata
host_tree <- keep.tip(host_tree, 
                      intersect(bac_metadata_css$sampleid, 
                                host_tree[["tip.label"]]))

host_plot <- ggtree(host_tree) + 
  geom_tiplab(size=2, align=TRUE, linesize=.5)

 combine Eunicea and core microbiome


In [ ]:
core_tree <- read.tree("/root/biommar/data/core_tree.nwk")

bac_core_taxa <- as.matrix(read.csv("/root/biommar/data/bac_core_taxa.tsv"))

# filter data to core microbiome members
core_tax <- taxa[rownames(taxa) %in% bac_core_taxa,]
core_feature <- bac_feature_table[,colnames(bac_feature_table) %in% 
                                        bac_core_taxa]
core_meta <- bac_metadata[rownames(bac_metadata) %in% 
                                rownames(core_feature),]

# reorder and subsitute tip labels to bear taxonomic labels
core_tree[["tip.label"]] <- core_tax[match(core_tree[["tip.label"]], 
                                           rownames(core_tax)),]$family

core_tree[["tip.label"]] <- as.character(core_tree[["tip.label"]])
core_feature <- as.data.frame(t(core_feature))


#core_feature <- core_feature[,match(get_taxa_name(host_plot), 
#                                   colnames(core_feature))]

core_feature$id <- rownames(core_feature)
molten <- melt(core_feature, id.vars=c("id"))
molten$sampleid <- molten$variable
data_merged <- merge(molten, core_meta[,c(1:12)], by="sampleid")

data_merged <- data_merged[data_merged$sampleid %in% host_tree[["tip.label"]],]

department data preparation 

In [ ]:
data_merged_department <- data_merged[order(data_merged[,"department"]),]
data_merged_department$sampleid <- 
  factor(data_merged_department$sampleid, 
         levels=unique((data_merged_department$sampleid)[order(
           data_merged_department[, "department"])]))

core_meta[, "department"] <- as.factor(core_meta[, "department"])
core_meta_department <- base::transform(core_meta, 
                                     count = table(department)[department])

core_meta_department <- core_meta_department[order(
  core_meta_department[, "department"]),]
core_feature_department <- core_feature[order(
  core_meta_department[, "department"]),]

names <- get_taxa_name(host_plot)

tmp_df <- data_merged_department$sampleid[match(rep(names, 
                                                    each=nrow(bac_core_taxa)), 
                                    data_merged_department$sampleid)]

tmp_df <- as.data.frame(tmp_df)
tmp_df$num <- rownames(tmp_df)
colnames(tmp_df) <- c("sampleid","num")

data_merged_department <- merge(data_merged_department, tmp_df, by="sampleid")
data_merged_department$num <- as.numeric(data_merged_department$num)

### department plotting 

In [ ]:
ph.tree <- ggtree(core_tree) + 
  geom_tiplab(size=2, align=TRUE, linesize=.5) +
  ggplot2::xlim(0, 1.2)

In [ ]:
ph.heat_department <- ggplot(data_merged_department) + 
  geom_tile(aes(x=id, y=as.character(reorder(sampleid,num)), fill=log10(value+1)), 
            color = "#000000") +
  scale_fill_viridis_c() + theme_void() + scale_y_discrete(limit=(names)) +
  coord_flip()

In [ ]:
ph.meta_department <- ggplot(data_merged_department) + 
  geom_raster(aes(y=1, x=sampleid, fill=department), alpha = 1) +
  theme_void() +
  scale_fill_manual(values=c("#FFC300","#FF5733","#C70039","#900C3F","#581845"))

In [ ]:
host_plot <- ggtree(host_tree) + 
ggplot2::xlim(-0.05, 0.05)
  
host_plot <- host_plot  %<+% core_meta + 
  layout_dendrogram() +
    geom_tippoint(aes(color=Clade)) +
    scale_color_manual(values=c("#00468BFF", "#ED0000FF", "#42B540FF", 
                                "#0099B4FF", "#925E9FFF", "#FDAF91FF", 
                                "#AD002AFF", "#ADB6B6FF", "#1B1919FF", 
                                "#374E55FF", "#DF8F44FF", "#900C3F", 
                                "#FFC300", "#79AF97FF", "green", "blue")) #+
  #geom_label(aes(label=node), size = 2) +
  #geom_cladelabel(node=194, label="mammosa complex", align=T, 
  #                angle=90, fontsize=3, color = "#AD002AFF", hjust = 0.2,
  #                offset.text=.001, barsize=1.5) +
  #
  #geom_cladelabel(node=215, label="pinta", align=T, 
  #                angle=90, fontsize=3 , color = "#1B1919FF", hjust = 0.2,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=222, label="flexuosa", align=T, 
  #                angle=90, fontsize=3, color = "#0099B4FF", hjust = 0.5,
  #                offset.text=.001, barsize=1.5) +
  #
  #geom_cladelabel(node=174, label="asperula - fusca", align=T, 
  #                angle=90, fontsize=3, color = "#00468BFF", hjust = 0.7,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=225, label="tayrona", align=T, 
  #                angle=90, fontsize=3, color = "#FFC300", hjust = 0.6,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=232, label="laciniata", align=T, 
  #                angle=90, fontsize=3, color = "#FDAF91FF", hjust = 0.2,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=256, label="knighti", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=267, label="calyclata", align=T, 
  #                angle=90, fontsize=3, color = "#ED0000FF", hjust = 0.5,
  #                offset.text=.001, barsize=1.5) +
  #
  #geom_cladelabel(node=268, label="tourneforti", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5) +
  #
  #geom_cladelabel(node=280, label="sp.", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=302, label="clavigera", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5) +
  #
  #geom_cladelabel(node=312, label="sp. varias", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5) +
  # 
  #geom_cladelabel(node=315, label="sp. curazao", align=T, 
  #                angle=90, fontsize=3, color =  "#925E9FFF", hjust = 0.3,
  #                offset.text=.001, barsize=1.5)
 
#161 - 157
#host_plot <- host_plot + geom_tiplab(angle=90, hjust=1, offset=-10, show.legend=FALSE)

plot_core_host <-
  ph.heat_department %>% insert_top(host_plot, 
                                   height = 0.5) %>% 
    insert_left(ph.tree, width=0.5) %>%  insert_bottom(ph.meta_department, 
                                                      height = 0.1)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8, repr.plot.res = 200)

In [ ]:
plot_core_host

<br><hr><br>
<a id='ALDEx2'></a>
# ALDEx2


[&#8593; back to content &#8593;](#Content)

In [ ]:
bac_feature_table <- read.delim("/root/biommar/data/B_asv.txt", check.names = F)
bac_metadata <- read.delim("/root/biommar/data/B_asv_metadata.txt")

deps <- unique(bac_metadata$department)
deps_comb <- combn(deps, 2)

for(i in 1:ncol(deps_comb)) {
  comb <- deps_comb[,i]
  cond <- bac_metadata$department[bac_metadata$department %in% comb]
  dat <- t(bac_feature_table[
      rownames(bac_feature_table) %in% 
        rownames(bac_metadata[bac_metadata$department %in% comb,])
      ,])
  z <- aldex(dat, cond, mc.samples=100, test="t", effect=TRUE,
        include.sample.summary=FALSE, denom="all", verbose=FALSE)
  assign(paste("x",i, collapse="", sep=""), z) 
}

<hr>

### Function: _aldex.ggplot_

ALDEx2 provides a plotting function that uses R base plots. The function below produces similar plots using the ggplot package. Some scaling has to be adjusted manually. In general, ggplot should provide more options for manipulation of plots.

In [ ]:
aldex.ggplot <- 
  function(x, ..., type = c("MW", "MA"), all.col = rgb(0, 0, 0, 0.2), 
           all.pch = 20,all.cex = 2, called.col = "red", called.pch = 20, 
           called.cex = 2.5, thres.line.col = "darkgrey", thres.lwd = 0.5, 
           test = "welch", cutoff.pval = 0.05, cutoff.effect = 1, 
           rare.col = "black", rare = 0, rare.pch = 20, rare.cex = 2){
  require(grid)
  require(ggplot2)
  called <- x$we.eBH <= cutoff.pval
  # names must not contain "."
  cond.1 <- strsplit(colnames(x)[2], ".", fixed=TRUE)[[1]][3] 
  cond.2 <- strsplit(colnames(x)[3], ".", fixed=TRUE)[[1]][3]
  
  if (type == "MW") {
  plot <- 
    ggplot() +
    theme_bw() +
    geom_abline(aes(intercept=0, slope=1), 
                color=thres.line.col, linetype=2, size=thres.lwd) +
    geom_abline(aes(intercept=0, slope=-1), 
                color=thres.line.col, linetype=2, size=thres.lwd) +
    geom_point(data=x, aes(x=diff.win, y=diff.btw),
               color=all.col, shape=all.pch, size=all.cex) +
    xlab(expression("Median" ~ ~Log[2] ~ ~"Dispersion")) +
    ylab(expression(atop("Median" ~ ~Log[2] ~ ~"Difference"))) +
    scale_color_manual(values=c(rare.col, called.col))
  if(nrow(x[x$rab.all < rare,]) > 0) {
    plot <- plot + 
      geom_point(data=x[x$rab.all < rare,], 
                 aes(x=diff.win, y=diff.btw), color=rare.col, 
                 shape=rare.pch, size=rare.cex)
  }
  if(nrow(x[called,]) > 0) {
    plot <- plot + 
      geom_point(data=x[called,], aes(x=diff.win, y=diff.btw), 
                 color=called.col, shape=called.pch, size=called.cex)
  }
  plot <- plot + ggtitle("")
  print(plot)
  grid.text(
    cond.1,
    rot = 90,
    gp=gpar(fontsize=8, col="gray", fontface="bold"),
    x=unit(0, "npc")+unit(7,'mm'),
    y=unit(0, "npc")+unit(12,'mm'),
    just=c("left", "top"))
  grid.text(
    cond.2,
    rot = 90,
    gp=gpar(fontsize=8, col="gray", fontface="bold"),
    x=unit(0, "npc")+unit(7,'mm'),
    y=unit(1, "npc")-unit(10,'mm'),
    just=c("right", "top"))
  } 
  if (type == "MA") {
    plot <- 
      ggplot() +
      theme_bw() +
      geom_point(data=x, aes(x=rab.all, y=diff.btw),
                 color=all.col, shape=all.pch, size=all.cex) +
      xlab(expression("Median" ~ ~Log[2] ~ ~"Relative Abundance")) +
      ylab(expression(atop("Median" ~ ~Log[2] ~ ~"Difference"))) +
      scale_color_manual(values=c(rare.col, called.col))
    if(nrow(x[x$rab.all < rare,]) > 0) {
      plot <- plot + 
        geom_point(data=x[x$rab.all < rare,], 
                   aes(x=rab.all, y=diff.btw), color=rare.col, 
                   shape=rare.pch, size=rare.cex)
    }
    if(nrow(x[called,]) > 0) {
      plot <- plot + 
        geom_point(data=x[called,], aes(x=rab.all, y=diff.btw), 
                   color=called.col, shape=called.pch, size=called.cex)
    }
    plot <- plot + ggtitle("")
    print(plot)
    grid.text(
      cond.1,
      rot = 90,
      gp=gpar(fontsize=8, col="gray", fontface="bold"),
      x=unit(0, "npc")+unit(7,'mm'),
      y=unit(0, "npc")+unit(12,'mm'),
      just=c("left", "top"))
    grid.text(
      cond.2,
      rot = 90,
      gp=gpar(fontsize=8, col="gray", fontface="bold"),
      x=unit(0, "npc")+unit(7,'mm'),
      y=unit(1, "npc")-unit(10,'mm'),
      just=c("right", "top"))
  } 
  }

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 4, repr.plot.res = 200)

<hr><br>
<h3>Cartagena vs. Curazao</h3>

In [ ]:
aldex.ggplot(x1, type="MW") 
aldex.ggplot(x1, type="MA")

<hr><br>
<h3>Cartagena vs. Long Keys Florida</h3>

In [ ]:
aldex.ggplot(x2, type="MW")
aldex.ggplot(x2, type="MA")

<hr><br>
<h3>Bocas del Toro vs. Cartagena</h3>

In [ ]:
aldex.ggplot(x3, type="MW")
aldex.ggplot(x3, type="MA")

<hr><br>
<h3>Cartagena vs. San Andres Isla</h3>

In [ ]:
aldex.ggplot(x4, type="MW")
aldex.ggplot(x4, type="MA")

<hr><br>
<h3>Curazao vs. Long Keys Florida</h3>

In [ ]:
aldex.ggplot(x5, type="MW")
aldex.ggplot(x5, type="MA")

<hr><br>
<h3>Bocas el Toro vs. Curazao</h3>

In [ ]:
aldex.ggplot(x6, type="MW")
aldex.ggplot(x6, type="MA")

<hr><br>
<h3>Curazao vs. San Andres Isla</h3>

In [ ]:
aldex.ggplot(x7, type="MW")
aldex.ggplot(x7, type="MA")

<hr><br>
<h3>Bocas del Toro vs. Long Keys Florida</h3>

In [ ]:
aldex.ggplot(x8, type="MW")
aldex.ggplot(x8, type="MA")

<hr><br>
<h3>Long Keys Florida vs. San Andres Isla</h3>

In [ ]:
aldex.ggplot(x9, type="MW")
aldex.ggplot(x9, type="MA")

<hr><br>
<h3>Bocas del Toro vs. San Andres Isla</h3>

In [ ]:
aldex.ggplot(x10, type="MW")
aldex.ggplot(x10, type="MA")

<br><hr><br>
### Save R Environment

Saving the objects in the R environment to a file lets you access them faster after restarting the notebook. 

In [ ]:
save.image("/root/biommar/data/environment.RData")

When closing the notebook and shutting down the kernel, objects in the environment are removed. To restore objects, run the following cells upon restart. 

In [ ]:
load("/root/biommar/data/environment.RData")

To see all objects in the current environment, run:

In [3]:
ls()

character(0)

<br><hr>
[&#8593; back to content &#8593;](#Content)
<hr><hr><hr><br>